# CELL 1: Installation & Environment Setup

In [5]:

import subprocess
import sys
import warnings
import os

print("="*80)
print("ENVIRONMENT SETUP - FINAL VERSION")
print("="*80)

# 1. Clean install
print("\n[1/4] Cleaning old packages...")
subprocess.run(["pip", "uninstall", "-y", "-q", 
                "scipy", "scikit-learn", "scikit-image", "pillow", "torchvision", "torch"], 
               stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)

# 2. Install PyTorch ONLY (no torchvision)
print("[2/4] Installing PyTorch 2.0.1...")
subprocess.run(["pip", "install", "--no-cache-dir", "-q", "torch==2.0.1"], 
               stderr=subprocess.DEVNULL)

# 3. Install NumPy & basics
print("[3/4] Installing NumPy, Pillow, OpenCV...")
subprocess.run(["pip", "install", "--no-cache-dir", "-q",
                "numpy==1.24.3",
                "pillow==10.0.0",
                "matplotlib==3.7.1",
                "seaborn==0.12.2",
                "pandas==2.0.3",
                "tqdm==4.66.1",
                "opencv-python==4.8.0.74",
                "scikit-image==0.21.0"],
               stderr=subprocess.DEVNULL)

# 4. Install SciPy
print("[4/4] Installing SciPy...")
subprocess.run(["pip", "install", "--no-cache-dir", "-q", "scipy==1.11.0"],
               stderr=subprocess.DEVNULL)

print("\n" + "="*80)
print("VERIFYING IMPORTS...")
print("="*80 + "\n")

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import PyTorch core
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

print(f"✓ PyTorch {torch.__version__}")

# Import data science libraries
import numpy as np
import pandas as pd
import scipy
import cv2
from PIL import Image

print(f"✓ NumPy {np.__version__}")
print(f"✓ Pandas {pd.__version__}")
print(f"✓ SciPy {scipy.__version__}")
print(f"✓ OpenCV available")
print(f"✓ PIL available")

# Import visualization
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

print(f"✓ Matplotlib {matplotlib.__version__}")
print(f"✓ Seaborn available")

# Import utilities
from tqdm import tqdm
import json
import time
import random
import gc
from collections import defaultdict
from pathlib import Path

print(f"✓ Utilities imported")

# Manual torchvision.transforms replacement using OpenCV
class transforms:
    """Manual transforms implementation using OpenCV (no torchvision dependency)"""
    
    class Normalize:
        def __init__(self, mean, std):
            self.mean = np.array(mean)
            self.std = np.array(std)
        
        def __call__(self, img):
            return (img - self.mean) / self.std
    
    class ToTensor:
        def __call__(self, img):
            if isinstance(img, np.ndarray):
                img = torch.from_numpy(img).float()
                if len(img.shape) == 2:
                    img = img.unsqueeze(0)
                elif img.shape[2] == 3:
                    img = img.permute(2, 0, 1)
                return img
            return torch.from_numpy(np.array(img)).float()
    
    class RandomHorizontalFlip:
        def __init__(self, p=0.5):
            self.p = p
        
        def __call__(self, img):
            if random.random() < self.p:
                img = cv2.flip(img, 1)
            return img
    
    class RandomVerticalFlip:
        def __init__(self, p=0.5):
            self.p = p
        
        def __call__(self, img):
            if random.random() < self.p:
                img = cv2.flip(img, 0)
            return img
    
    class RandomResizedCrop:
        def __init__(self, size, scale=(0.08, 1.0), ratio=(3./4., 4./3.)):
            self.size = size
            self.scale = scale
            self.ratio = ratio
        
        def __call__(self, img):
            h, w = img.shape[:2]
            area = h * w
            
            for _ in range(10):
                target_area = area * random.uniform(*self.scale)
                log_ratio = (np.log(self.ratio[0]), np.log(self.ratio[1]))
                aspect_ratio = np.exp(random.uniform(*log_ratio))
                
                w_crop = int(np.sqrt(target_area * aspect_ratio))
                h_crop = int(np.sqrt(target_area / aspect_ratio))
                
                if w_crop < w and h_crop < h:
                    top = random.randint(0, h - h_crop)
                    left = random.randint(0, w - w_crop)
                    img = img[top:top+h_crop, left:left+w_crop]
                    img = cv2.resize(img, (self.size, self.size))
                    return img
            
            return cv2.resize(img, (self.size, self.size))
    
    class ColorJitter:
        def __init__(self, brightness=0, contrast=0, saturation=0, hue=0):
            self.brightness = brightness
            self.contrast = contrast
            self.saturation = saturation
            self.hue = hue
        
        def __call__(self, img):
            # Simple brightness adjustment
            if self.brightness > 0:
                factor = random.uniform(1-self.brightness, 1+self.brightness)
                img = np.clip(img * factor, 0, 255).astype(np.uint8)
            return img
    
    class GaussianBlur:
        def __init__(self, kernel_size=5, sigma=(0.1, 2.0)):
            self.kernel_size = kernel_size
            self.sigma = sigma
        
        def __call__(self, img):
            if random.random() < 0.5:
                sig = random.uniform(*self.sigma)
                img = cv2.GaussianBlur(img, (self.kernel_size, self.kernel_size), sig)
            return img
    
    class RandomRotation:
        def __init__(self, degrees=15):
            self.degrees = degrees
        
        def __call__(self, img):
            if random.random() < 0.5:
                angle = random.uniform(-self.degrees, self.degrees)
                h, w = img.shape[:2]
                M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
                img = cv2.warpAffine(img, M, (w, h))
            return img
    
    class Compose:
        def __init__(self, transforms_list):
            self.transforms = transforms_list
        
        def __call__(self, img):
            for t in self.transforms:
                img = t(img)
            return img

# Manual ResNet50 loading (just use torch's built-in)
def resnet50(pretrained=True):
    """Load ResNet50 backbone"""
    try:
        # Try using torchvision if available
        from torchvision.models import resnet50 as tv_resnet50
        return tv_resnet50(pretrained=pretrained)
    except:
        # Fallback: create empty ResNet50 without weights
        print("⚠ Using ResNet50 without pretrained weights (torchvision unavailable)")
        # For now, we'll create a simple backbone
        raise ImportError("Please install torchvision separately")

print(f"✓ Manual transforms implementation loaded")

# Set reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"\n✓ CUDA Available: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print(f"\n⚠ CPU mode (no CUDA)")

print(f"✓ Device: {str(device).upper()}")

print("\n" + "="*80)
print("✓✓✓ ENVIRONMENT READY ✓✓✓")
print("="*80 + "\n")


ENVIRONMENT SETUP - FINAL VERSION

[1/4] Cleaning old packages...
[2/4] Installing PyTorch 2.0.1...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 366.4 MB/s eta 0:00:00
[3/4] Installing NumPy, Pillow, OpenCV...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 150.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 279.5 MB/s eta 0:00:00
[4/4] Installing SciPy...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# CELL 2: DATASET SETUP & LOADING

In [8]:
# CELL 2: DATASET SETUP & LOADING

import os
from pathlib import Path

# Setup directories
DATA_DIR = "/kaggle/input/dataset-for-classifying-rice-varieties-in-bd/Rice Varieties in Bangladesh/Original" 
OUTPUT_DIR = "/kaggle/working"
checkpoint_dir = os.path.join(OUTPUT_DIR, "checkpoints")

os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*80)
print("DATASET SETUP")
print("="*80 + "\n")

# Get all class names from directory
class_names = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
num_classes = len(class_names)

print(f"Classes found: {num_classes}")
print(f"Classes: {class_names}\n")

# Count images per class
class_counts = {}
for cls in class_names:
    cls_path = os.path.join(DATA_DIR, cls)
    count = len([f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    class_counts[cls] = count
    print(f"  {cls}: {count} images")

total_images = sum(class_counts.values())
print(f"\nTotal images: {total_images}\n")

print("="*80)
print("✓ Dataset ready\n")


DATASET SETUP

Classes found: 38
Classes: ['BD30', 'BD33', 'BD39', 'BD49', 'BD51', 'BD52', 'BD56', 'BD57', 'BD70', 'BD72', 'BD75', 'BD76', 'BD79', 'BD85', 'BD87', 'BD91', 'BD93', 'BD95', 'BR22', 'BR23', 'BRRI102', 'BRRI67', 'BRRI74', 'Binadhan10', 'Binadhan11', 'Binadhan12', 'Binadhan14', 'Binadhan16', 'Binadhan17', 'Binadhan19', 'Binadhan20', 'Binadhan21', 'Binadhan23', 'Binadhan24', 'Binadhan25', 'Binadhan26', 'Binadhan7', 'Binadhan8']

  BD30: 500 images
  BD33: 500 images
  BD39: 500 images
  BD49: 500 images
  BD51: 500 images
  BD52: 500 images
  BD56: 500 images
  BD57: 500 images
  BD70: 500 images
  BD72: 500 images
  BD75: 500 images
  BD76: 500 images
  BD79: 500 images
  BD85: 500 images
  BD87: 500 images
  BD91: 500 images
  BD93: 500 images
  BD95: 500 images
  BR22: 500 images
  BR23: 500 images
  BRRI102: 500 images
  BRRI67: 500 images
  BRRI74: 500 images
  Binadhan10: 500 images
  Binadhan11: 500 images
  Binadhan12: 500 images
  Binadhan14: 500 images
  Binadhan16:

# CELL 3: CUSTOM DATASET & DATA AUGMENTATION

In [9]:
# CELL 3: CUSTOM DATASET & DATA AUGMENTATION

class RiceDataset(Dataset):
    def __init__(self, data_dir, class_names, transform=None, img_size=224):
        self.data_dir = data_dir
        self.class_names = class_names
        self.transform = transform
        self.img_size = img_size
        self.images = []
        self.labels = []
        
        # Load all images
        for class_idx, class_name in enumerate(class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_file in os.listdir(class_dir):
                if img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
                    self.images.append(os.path.join(class_dir, img_file))
                    self.labels.append(class_idx)
        
        print(f"Loaded {len(self.images)} images")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        # Load image
        try:
            img = cv2.imread(img_path)
            if img is None:
                img = cv2.imread(self.images[0])
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = img.astype(np.float32) / 255.0
            
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            img = np.random.rand(self.img_size, self.img_size, 3).astype(np.float32)
        
        if self.transform:
            img = self.transform(img)
        
        # Convert to tensor
        img = torch.from_numpy(img.transpose(2, 0, 1)).float()
        
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }


class AugmentationPipeline:
    def __init__(self, img_size=224):
        self.img_size = img_size
    
    def __call__(self, img):
        # Random horizontal flip
        if random.random() < 0.5:
            img = cv2.flip(img, 1)
        
        # Random vertical flip
        if random.random() < 0.3:
            img = cv2.flip(img, 0)
        
        # Random rotation
        if random.random() < 0.5:
            angle = random.uniform(-15, 15)
            h, w = img.shape[:2]
            M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
            img = cv2.warpAffine(img, M, (w, h))
        
        # Random brightness
        if random.random() < 0.5:
            brightness_factor = random.uniform(0.85, 1.15)
            img = np.clip(img * brightness_factor, 0, 1)
        
        # Random contrast
        if random.random() < 0.5:
            contrast_factor = random.uniform(0.85, 1.15)
            img = np.clip(img * contrast_factor, 0, 1)
        
        # Random Gaussian blur
        if random.random() < 0.3:
            img = cv2.GaussianBlur(img, (5, 5), 0)
        
        return img


print("="*80)
print("CREATING DATASETS")
print("="*80 + "\n")

# Create datasets
train_transform = AugmentationPipeline(img_size=224)
train_dataset = RiceDataset(DATA_DIR, class_names, transform=train_transform, img_size=224)

print(f"✓ Dataset created: {len(train_dataset)} images, {num_classes} classes\n")

# DataLoader
BATCH_SIZE = 32
NUM_WORKERS = 4

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"✓ DataLoader created")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Batches per epoch: {len(train_loader)}")
print(f"  Total samples: {len(train_dataset)}\n")

print("="*80)
print("✓ Data loading complete\n")


CREATING DATASETS

Loaded 19000 images
✓ Dataset created: 19000 images, 38 classes

✓ DataLoader created
  Batch size: 32
  Batches per epoch: 594
  Total samples: 19000

✓ Data loading complete



# CELL 4: DINO MODEL IMPLEMENTATION


In [10]:
# CELL 4: DINO MODEL IMPLEMENTATION

class DINO_Backbone(nn.Module):
    def __init__(self, img_size=224, patch_size=16, embed_dim=384, depth=12, num_heads=12):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.embed_dim = embed_dim
        
        # Patch embedding
        self.patch_embed = nn.Conv2d(3, embed_dim, kernel_size=patch_size, stride=patch_size)
        
        # Class token
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        
        # Position embedding
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, embed_dim))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
        # Transformer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=embed_dim * 4,
            dropout=0.1,
            batch_first=True,
            activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        
        # Layer norm
        self.norm = nn.LayerNorm(embed_dim)
        
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
    
    def forward(self, x):
        B = x.shape[0]
        
        # Patch embedding
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2)
        
        # Add class token
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        
        # Add position embedding
        x = x + self.pos_embed
        
        # Transformer
        x = self.transformer(x)
        x = self.norm(x)
        
        return x[:, 0]


class DINOHead(nn.Module):
    def __init__(self, in_dim=384, out_dim=4096, hidden_dim=2048, num_layers=3):
        super().__init__()
        layers = []
        for i in range(num_layers):
            if i == 0:
                layers.append(nn.Linear(in_dim, hidden_dim))
            elif i == num_layers - 1:
                layers.append(nn.Linear(hidden_dim, out_dim))
            else:
                layers.append(nn.Linear(hidden_dim, hidden_dim))
            
            if i < num_layers - 1:
                layers.append(nn.GELU())
                layers.append(nn.BatchNorm1d(hidden_dim))
        
        self.mlp = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.mlp(x)


class DINOModel(nn.Module):
    def __init__(self, backbone_dim=384, proj_dim=4096):
        super().__init__()
        self.backbone = DINO_Backbone(embed_dim=backbone_dim)
        self.student_head = DINOHead(in_dim=backbone_dim, out_dim=proj_dim)
        self.teacher_backbone = DINO_Backbone(embed_dim=backbone_dim)
        self.teacher_head = DINOHead(in_dim=backbone_dim, out_dim=proj_dim)
        
        # Freeze teacher
        for p in self.teacher_backbone.parameters():
            p.requires_grad = False
        for p in self.teacher_head.parameters():
            p.requires_grad = False
    
    def forward(self, x_student, x_teacher):
        # Student forward
        student_feat = self.backbone(x_student)
        student_out = self.student_head(student_feat)
        
        # Teacher forward (no grad)
        with torch.no_grad():
            teacher_feat = self.teacher_backbone(x_teacher)
            teacher_out = self.teacher_head(teacher_feat)
        
        return student_out, teacher_out, student_feat


print("="*80)
print("DINO MODEL ARCHITECTURE")
print("="*80 + "\n")

# Model params
EMBED_DIM = 384
PROJ_DIM = 4096
BACKBONE_DIM = 384

# Create model
dino_model = DINOModel(backbone_dim=BACKBONE_DIM, proj_dim=PROJ_DIM).to(device)

print(f"✓ DINO Model created")
print(f"  Backbone: Vision Transformer")
print(f"  Embedding dim: {EMBED_DIM}")
print(f"  Projection dim: {PROJ_DIM}")
print(f"  Device: {device}\n")

# Count parameters
total_params = sum(p.numel() for p in dino_model.parameters())
student_params_count = sum(p.numel() for p in dino_model.backbone.parameters()) + sum(p.numel() for p in dino_model.student_head.parameters())

print(f"  Total parameters: {total_params:,}")
print(f"  Student parameters (trainable): {student_params_count:,}\n")

print("="*80)
print("✓ Model ready\n")


DINO MODEL ARCHITECTURE

✓ DINO Model created
  Backbone: Vision Transformer
  Embedding dim: 384
  Projection dim: 4096
  Device: cuda

  Total parameters: 70,102,784
  Student parameters (trainable): 35,051,392

✓ Model ready



# CELL 5: TRAINING SETUP & LOSS FUNCTION


In [13]:
# CELL 5: TRAINING SETUP & LOSS FUNCTION (PATCHED)

# CRITICAL FIX: Disable ONNX imports manually to prevent PyTorch crash
import sys
import types
# Mock the missing ONNX module to bypass the error
sys.modules['torch.onnx'] = types.ModuleType('torch.onnx')
sys.modules['torch.onnx'].is_in_onnx_export = False

class DINOLoss(nn.Module):
    def __init__(self, out_dim=4096, warmup_teacher_temp=0.04, teacher_temp=0.07, student_temp=0.1):
        super().__init__()
        self.student_temp = student_temp
        self.teacher_temp = teacher_temp
        self.center = torch.zeros(1, out_dim).to(device)
    
    def forward(self, student_output, teacher_output):
        # Normalize
        student_output = torch.nn.functional.normalize(student_output, dim=-1, p=2)
        teacher_output = torch.nn.functional.normalize(teacher_output, dim=-1, p=2)
        
        # DINO loss: cross entropy with temperature
        student_out = student_output / self.student_temp
        teacher_out = teacher_output / self.teacher_temp
        
        # Compute cross-entropy
        loss = torch.mean(torch.sum(-teacher_out * torch.nn.functional.log_softmax(student_out, dim=-1), dim=-1))
        
        return loss


print("="*80)
print("TRAINING SETUP (PATCHED)")
print("="*80 + "\n")

# Training params
NUM_EPOCHS = 35
LEARNING_RATE = 0.02
MOMENTUM = 0.9
WARMUP_EPOCHS = 3
WEIGHT_DECAY = 1e-4

# Optimizer (using SGD)
student_params_list = list(dino_model.backbone.parameters()) + list(dino_model.student_head.parameters())

# Use raw SGD implementation if torch.optim fails
try:
    optimizer = torch.optim.SGD(student_params_list, lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
except Exception as e:
    print(f"⚠ Standard optimizer failed, using minimal custom SGD: {e}")
    # Minimal SGD implementation as fallback
    class MinimalSGD:
        def __init__(self, params, lr=0.01, momentum=0.9, weight_decay=0):
            self.params = list(params)
            self.lr = lr
            self.momentum = momentum
            self.weight_decay = weight_decay
            self.velocities = [torch.zeros_like(p) for p in self.params]
            self.param_groups = [{'lr': lr}] # Compatibility
        
        def zero_grad(self):
            for p in self.params:
                if p.grad is not None:
                    p.grad.detach_()
                    p.grad.zero_()
        
        def step(self):
            with torch.no_grad():
                for i, p in enumerate(self.params):
                    if p.grad is None: continue
                    grad = p.grad
                    if self.weight_decay != 0:
                        grad = grad.add(p, alpha=self.weight_decay)
                    
                    vel = self.velocities[i]
                    vel.mul_(self.momentum).add_(grad)
                    p.add_(vel, alpha=-self.lr)
        
        def state_dict(self): return {} # Dummy
        def load_state_dict(self, state_dict): pass # Dummy

    optimizer = MinimalSGD(student_params_list, lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)


# Loss
criterion = DINOLoss(out_dim=PROJ_DIM).to(device)

# Learning rate scheduler
try:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-5)
except:
    # Fallback simple scheduler
    class SimpleScheduler:
        def __init__(self, optimizer): self.optimizer = optimizer
        def step(self): pass
    scheduler = SimpleScheduler(optimizer)

print(f"✓ Training configured")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Base learning rate: {LEARNING_RATE}")
print(f"  Optimizer: SGD (patched)")
print(f"  Loss: DINO Cross-Entropy\n")

print("="*80)
print("✓ Setup complete\n")


TRAINING SETUP (PATCHED)

⚠ Standard optimizer failed, using minimal custom SGD: cannot import name 'set_guard_fail_hook' from 'torch._dynamo.eval_frame' (/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py)
✓ Training configured
  Epochs: 35
  Base learning rate: 0.02
  Optimizer: SGD (patched)
  Loss: DINO Cross-Entropy

✓ Setup complete



# CELL 6: TRAINING LOOP

In [14]:
# CELL 6: TRAINING LOOP (35 EPOCHS)

print("="*80)
print("DINO PRETRAINING STARTED")
print("="*80 + "\n")

train_losses = []
best_loss = float('inf')
patience = 7
patience_counter = 0
start_time = time.time()

# EMA (Exponential Moving Average) for teacher
EMA_TAU = 0.999

for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0
    epoch_samples = 0
    
    dino_model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=False)
    
    for batch_idx, batch in enumerate(pbar):
        # Get images
        images = batch['image'].to(device)
        B = images.shape[0]
        
        # Create two augmented crops
        crop1 = images
        crop2 = images.clone()
        
        # Random augmentation on crop2
        if random.random() < 0.5:
            crop2 = torch.flip(crop2, [3])
        
        # Forward pass
        student_out, teacher_out, student_feat = dino_model(crop1, crop2)
        
        # Loss
        loss = criterion(student_out, teacher_out)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(student_params_list, max_norm=1.0)
        
        optimizer.step()
        
        # Update teacher (exponential moving average)
        with torch.no_grad():
            for param_s, param_t in zip(dino_model.backbone.parameters(), dino_model.teacher_backbone.parameters()):
                param_t.data = EMA_TAU * param_t.data + (1 - EMA_TAU) * param_s.data.detach()
            
            for param_s, param_t in zip(dino_model.student_head.parameters(), dino_model.teacher_head.parameters()):
                param_t.data = EMA_TAU * param_t.data + (1 - EMA_TAU) * param_s.data.detach()
        
        epoch_loss += loss.item() * B
        epoch_samples += B
        
        pbar.set_postfix({'loss': f'{epoch_loss/max(epoch_samples, 1):.4f}'})
    
    # Epoch metrics
    avg_loss = epoch_loss / epoch_samples
    train_losses.append(avg_loss)
    
    # Learning rate update
    try:
        scheduler.step()
    except:
        pass
    
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Loss: {avg_loss:.4f} | LR: {current_lr:.6f}")
    
    # Save checkpoint
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'student_state': dino_model.backbone.state_dict(),
            'teacher_state': dino_model.teacher_backbone.state_dict(),
            'optimizer_state': optimizer.param_groups if hasattr(optimizer, 'param_groups') else {},
            'loss': avg_loss,
            'train_losses': train_losses,
        }, os.path.join(checkpoint_dir, 'best_checkpoint.pt'))
        print(f"  ✓ Saved best checkpoint (loss: {avg_loss:.4f})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\n✓ Early stopping triggered after {patience} epochs without improvement")
            break

total_training_time = (time.time() - start_time) / 3600

print(f"\n{'='*80}")
print(f"TRAINING COMPLETED")
print(f"{'='*80}")
print(f"✓ Total time: {total_training_time:.2f} hours")
print(f"✓ Best loss: {best_loss:.4f}")
print(f"✓ Epochs trained: {len(train_losses)}")
print(f"✓ Checkpoint saved to {checkpoint_dir}\n")

# Save training losses for visualization
np.save(os.path.join(OUTPUT_DIR, 'train_losses.npy'), np.array(train_losses))
print(f"✓ Training losses saved\n")


DINO PRETRAINING STARTED



Epoch 1/35 | Loss: -43.0315 | LR: 0.020000
  ✓ Saved best checkpoint (loss: -43.0315)


Epoch 2/35 | Loss: -121.6097 | LR: 0.020000
  ✓ Saved best checkpoint (loss: -121.6097)


Epoch 3/35 | Loss: -143.6975 | LR: 0.020000
  ✓ Saved best checkpoint (loss: -143.6975)


Epoch 4/35 | Loss: -143.4943 | LR: 0.020000


...

Epoch 6/35 | Loss: -142.9534 | LR: 0.020000


Epoch 7/35 | Loss: -142.8529 | LR: 0.020000


Epoch 8/35 | Loss: -142.7935 | LR: 0.020000


Epoch 9/35 | Loss: -142.7681 | LR: 0.020000


Epoch 10/35 | Loss: -142.7732 | LR: 0.020000

✓ Early stopping triggered after 7 epochs without improvement

TRAINING COMPLETED
✓ Total time: 0.55 hours
✓ Best loss: -143.6975
✓ Epochs trained: 10
✓ Checkpoint saved to /kaggle/working/checkpoints

✓ Training losses saved



# CELL 7: FEATURE EXTRACTION


In [15]:
# CELL 7: FEATURE EXTRACTION

print("="*80)
print("FEATURE EXTRACTION")
print("="*80 + "\n")

# Load best model
best_checkpoint = torch.load(os.path.join(checkpoint_dir, 'best_checkpoint.pt'), map_location=device)
dino_model.backbone.load_state_dict(best_checkpoint['student_state'])
print(f"✓ Loaded best student checkpoint (loss: {best_checkpoint['loss']:.4f})\n")

# Extract features
def extract_features(model, dataloader, device):
    model.eval()
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting features"):
            images = batch['image'].to(device)
            labels = batch['label']
            
            features = model.backbone(images)
            
            all_features.append(features.cpu().numpy())
            all_labels.append(labels.numpy())
    
    features = np.concatenate(all_features, axis=0)
    labels = np.concatenate(all_labels, axis=0)
    
    return features, labels

print("Extracting training features...")
train_features, train_labels = extract_features(dino_model, train_loader, device)

print(f"\n✓ Extracted features successfully")
print(f"  Feature shape: {train_features.shape}")
print(f"  Feature dimension: {train_features.shape[1]}")
print(f"  Number of samples: {train_features.shape[0]}")
print(f"  Label shape: {train_labels.shape}")
print(f"  Unique classes: {len(np.unique(train_labels))}\n")

# Save features
np.save(os.path.join(OUTPUT_DIR, 'train_features.npy'), train_features)
np.save(os.path.join(OUTPUT_DIR, 'train_labels.npy'), train_labels)
print(f"✓ Features saved to {OUTPUT_DIR}\n")

print("="*80)
print("✓ Feature extraction complete\n")


FEATURE EXTRACTION

✓ Loaded best student checkpoint (loss: -143.6975)

Extracting training features...


Extracting features: 100%|██████████| 594/594 [01:26<00:00,  6.90it/s]


✓ Extracted features successfully
  Feature shape: (19000, 384)
  Feature dimension: 384
  Number of samples: 19000
  Label shape: (19000,)
  Unique classes: 38

✓ Features saved to /kaggle/working

✓ Feature extraction complete



# CELL 8: LINEAR PROBE EVALUATION


In [16]:
# CELL 8: LINEAR PROBE EVALUATION

print("="*80)
print("LINEAR PROBE TRAINING")
print("="*80 + "\n")

# Manual train-test split
def manual_train_test_split(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    split_idx = int(len(X) * (1 - test_size))
    train_idx = indices[:split_idx]
    test_idx = indices[split_idx:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train_lp, X_val_lp, y_train_lp, y_val_lp = manual_train_test_split(train_features, train_labels, test_size=0.2)

print(f"Linear Probe split:")
print(f"  Train: {X_train_lp.shape[0]} samples")
print(f"  Val: {X_val_lp.shape[0]} samples\n")

# Linear probe classifier
class LinearProbe(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.fc(x)

lr_model = LinearProbe(X_train_lp.shape[1], num_classes).to(device)

# Convert to tensors
X_train_tensor = torch.from_numpy(X_train_lp).float().to(device)
y_train_tensor = torch.from_numpy(y_train_lp).long().to(device)
X_val_tensor = torch.from_numpy(X_val_lp).float().to(device)
y_val_tensor = torch.from_numpy(y_val_lp).long()

# Create dataloader
lr_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
lr_loader_lp = torch.utils.data.DataLoader(lr_dataset, batch_size=128, shuffle=True)

# Loss and optimizer
lr_criterion = nn.CrossEntropyLoss()

# Use custom optimizer if needed
try:
    lr_optimizer = torch.optim.Adam(lr_model.parameters(), lr=0.01)
except:
    class MinimalAdam:
        def __init__(self, params, lr=0.01):
            self.params = list(params)
            self.lr = lr
            self.t = 0
            self.m = [torch.zeros_like(p) for p in self.params]
            self.v = [torch.zeros_like(p) for p in self.params]
            self.param_groups = [{'lr': lr}]
        
        def zero_grad(self):
            for p in self.params:
                if p.grad is not None:
                    p.grad.zero_()
        
        def step(self):
            self.t += 1
            with torch.no_grad():
                for i, p in enumerate(self.params):
                    if p.grad is None: continue
                    grad = p.grad
                    self.m[i] = 0.9 * self.m[i] + 0.1 * grad
                    self.v[i] = 0.999 * self.v[i] + 0.001 * grad**2
                    m_hat = self.m[i] / (1 - 0.9**self.t)
                    v_hat = self.v[i] / (1 - 0.999**self.t)
                    p.add_(m_hat / (torch.sqrt(v_hat) + 1e-8), alpha=-self.lr)
    
    lr_optimizer = MinimalAdam(lr_model.parameters(), lr=0.01)

# Train linear probe
print("Training linear probe...")
for epoch in range(50):
    lr_model.train()
    for features, labels in lr_loader_lp:
        lr_optimizer.zero_grad()
        logits = lr_model(features)
        loss = lr_criterion(logits, labels)
        loss.backward()
        lr_optimizer.step()

# Evaluate
lr_model.eval()
with torch.no_grad():
    val_logits = lr_model(X_val_tensor.to(device))
    y_val_pred = val_logits.argmax(dim=1).cpu().numpy()

# Metrics
linear_probe_acc = np.mean(y_val_pred == y_val_lp)
linear_probe_precision = np.mean([np.mean(y_val_pred[y_val_lp == c] == c) for c in range(num_classes) if np.sum(y_val_lp == c) > 0])
linear_probe_recall = np.mean([np.mean(y_val_lp[y_val_lp == c] == y_val_pred[y_val_lp == c]) for c in range(num_classes) if np.sum(y_val_lp == c) > 0])
linear_probe_f1 = 2 * (linear_probe_precision * linear_probe_recall) / (linear_probe_precision + linear_probe_recall + 1e-8)

print(f"\n✓ Linear Probe Results:")
print(f"  Accuracy:  {linear_probe_acc:.4f}")
print(f"  Precision: {linear_probe_precision:.4f}")
print(f"  Recall:    {linear_probe_recall:.4f}")
print(f"  F1-Score:  {linear_probe_f1:.4f}\n")

print("="*80)
print("✓ Linear probe evaluation complete\n")


LINEAR PROBE TRAINING

Linear Probe split:
  Train: 15200 samples
  Val: 3800 samples

Training linear probe...

✓ Linear Probe Results:
  Accuracy:  0.2887
  Precision: 0.2862
  Recall:    0.2862
  F1-Score:  0.2862

✓ Linear probe evaluation complete



# CELL 9: MLP CLASSIFIER


In [17]:
# CELL 9: MLP CLASSIFIER

print("="*80)
print("MLP CLASSIFIER TRAINING")
print("="*80 + "\n")

class SimpleMLPClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=1024):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, num_classes)
        )
    
    def forward(self, x):
        return self.mlp(x)

mlp = SimpleMLPClassifier(X_train_lp.shape[1], num_classes, hidden_dim=1024).to(device)

# Loss and optimizer
mlp_criterion = nn.CrossEntropyLoss()

try:
    mlp_optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
except:
    class MinimalAdam:
        def __init__(self, params, lr=0.001):
            self.params = list(params)
            self.lr = lr
            self.t = 0
            self.m = [torch.zeros_like(p) for p in self.params]
            self.v = [torch.zeros_like(p) for p in self.params]
            self.param_groups = [{'lr': lr}]
        
        def zero_grad(self):
            for p in self.params:
                if p.grad is not None:
                    p.grad.zero_()
        
        def step(self):
            self.t += 1
            with torch.no_grad():
                for i, p in enumerate(self.params):
                    if p.grad is None: continue
                    grad = p.grad
                    self.m[i] = 0.9 * self.m[i] + 0.1 * grad
                    self.v[i] = 0.999 * self.v[i] + 0.001 * grad**2
                    m_hat = self.m[i] / (1 - 0.9**self.t)
                    v_hat = self.v[i] / (1 - 0.999**self.t)
                    p.add_(m_hat / (torch.sqrt(v_hat) + 1e-8), alpha=-self.lr)
    
    mlp_optimizer = MinimalAdam(mlp.parameters(), lr=0.001)

# Create dataloader for MLP
mlp_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
mlp_loader = torch.utils.data.DataLoader(mlp_dataset, batch_size=128, shuffle=True)

# Train MLP
print("Training MLP classifier...")
best_mlp_loss = float('inf')
patience_mlp = 5
patience_counter_mlp = 0

for epoch in range(50):
    mlp.train()
    total_loss = 0
    for features, labels in mlp_loader:
        mlp_optimizer.zero_grad()
        logits = mlp(features)
        loss = mlp_criterion(logits, labels)
        loss.backward()
        mlp_optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(mlp_loader)
    
    if avg_loss < best_mlp_loss:
        best_mlp_loss = avg_loss
        patience_counter_mlp = 0
    else:
        patience_counter_mlp += 1
        if patience_counter_mlp >= patience_mlp:
            break
    
    if (epoch + 1) % 10 == 0:
        print(f"  Epoch {epoch+1}/50 - Loss: {avg_loss:.4f}")

# Evaluate MLP
mlp.eval()
with torch.no_grad():
    mlp_val_logits = mlp(X_val_tensor.to(device))
    mlp_val_pred = mlp_val_logits.argmax(dim=1).cpu().numpy()

mlp_acc = np.mean(mlp_val_pred == y_val_lp)
mlp_precision = np.mean([np.mean(mlp_val_pred[y_val_lp == c] == c) for c in range(num_classes) if np.sum(y_val_lp == c) > 0])
mlp_recall = np.mean([np.mean(y_val_lp[y_val_lp == c] == mlp_val_pred[y_val_lp == c]) for c in range(num_classes) if np.sum(y_val_lp == c) > 0])
mlp_f1 = 2 * (mlp_precision * mlp_recall) / (mlp_precision + mlp_recall + 1e-8)

print(f"\n✓ MLP Classifier Results:")
print(f"  Accuracy:  {mlp_acc:.4f}")
print(f"  Precision: {mlp_precision:.4f}")
print(f"  Recall:    {mlp_recall:.4f}")
print(f"  F1-Score:  {mlp_f1:.4f}\n")

print("="*80)
print("✓ MLP evaluation complete\n")


MLP CLASSIFIER TRAINING

Training MLP classifier...
  Epoch 10/50 - Loss: 2.2375
  Epoch 20/50 - Loss: 2.0587
  Epoch 30/50 - Loss: 2.0217
  Epoch 40/50 - Loss: 1.9970

✓ MLP Classifier Results:
  Accuracy:  0.3582
  Precision: 0.3598
  Recall:    0.3598
  F1-Score:  0.3598

✓ MLP evaluation complete



# CELL 10: SVM CLASSIFIER (Manual Implementation)

In [18]:
# CELL 10: SVM CLASSIFIER (Manual Implementation)

print("="*80)
print("SVM CLASSIFIER TRAINING")
print("="*80 + "\n")

class SimpleSVM(nn.Module):
    def __init__(self, input_dim, num_classes, C=0.01):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        self.C = C
    
    def forward(self, x):
        return self.fc(x)
    
    def hinge_loss(self, output, target):
        one_hot = torch.zeros_like(output)
        one_hot.scatter_(1, target.unsqueeze(1), 1)
        
        margin = 1.0 - (output * one_hot - output * (1 - one_hot))
        loss = torch.clamp(margin, min=0).sum(dim=1).mean()
        
        l2_reg = torch.sum(self.fc.weight ** 2)
        return loss + self.C * l2_reg

svm_model = SimpleSVM(X_train_lp.shape[1], num_classes, C=0.01).to(device)

try:
    svm_optimizer = torch.optim.SGD(svm_model.parameters(), lr=0.01, momentum=0.9)
except:
    class MinimalSGD:
        def __init__(self, params, lr=0.01, momentum=0.9):
            self.params = list(params)
            self.lr = lr
            self.momentum = momentum
            self.velocities = [torch.zeros_like(p) for p in self.params]
            self.param_groups = [{'lr': lr}]
        
        def zero_grad(self):
            for p in self.params:
                if p.grad is not None:
                    p.grad.zero_()
        
        def step(self):
            with torch.no_grad():
                for i, p in enumerate(self.params):
                    if p.grad is None: continue
                    vel = self.velocities[i]
                    vel.mul_(self.momentum).add_(p.grad)
                    p.add_(vel, alpha=-self.lr)
    
    svm_optimizer = MinimalSGD(svm_model.parameters(), lr=0.01, momentum=0.9)

# Train SVM
print("Training SVM classifier...")
for epoch in range(50):
    svm_model.train()
    total_loss = 0
    for features, labels in mlp_loader:
        svm_optimizer.zero_grad()
        logits = svm_model(features)
        loss = svm_model.hinge_loss(logits, labels)
        loss.backward()
        svm_optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"  Epoch {epoch+1}/50 - Loss: {total_loss/len(mlp_loader):.4f}")

# Evaluate SVM
svm_model.eval()
with torch.no_grad():
    svm_val_logits = svm_model(X_val_tensor.to(device))
    svm_val_pred = svm_val_logits.argmax(dim=1).cpu().numpy()

svm_acc = np.mean(svm_val_pred == y_val_lp)
svm_precision = np.mean([np.mean(svm_val_pred[y_val_lp == c] == c) for c in range(num_classes) if np.sum(y_val_lp == c) > 0])
svm_recall = np.mean([np.mean(y_val_lp[y_val_lp == c] == svm_val_pred[y_val_lp == c]) for c in range(num_classes) if np.sum(y_val_lp == c) > 0])
svm_f1 = 2 * (svm_precision * svm_recall) / (svm_precision + svm_recall + 1e-8)

print(f"\n✓ SVM Classifier Results:")
print(f"  Accuracy:  {svm_acc:.4f}")
print(f"  Precision: {svm_precision:.4f}")
print(f"  Recall:    {svm_recall:.4f}")
print(f"  F1-Score:  {svm_f1:.4f}\n")

print("="*80)
print("✓ SVM evaluation complete\n")


SVM CLASSIFIER TRAINING

Training SVM classifier...
  Epoch 10/50 - Loss: 3.6463
  Epoch 20/50 - Loss: 3.6320
  Epoch 30/50 - Loss: 3.6171
  Epoch 40/50 - Loss: 3.5840
  Epoch 50/50 - Loss: 3.3947

✓ SVM Classifier Results:
  Accuracy:  0.1079
  Precision: 0.1044
  Recall:    0.1044
  F1-Score:  0.1044

✓ SVM evaluation complete



# CELL 11: CONFUSION MATRIX & PER-CLASS ANALYSIS


In [19]:
# CELL 11: CONFUSION MATRIX & PER-CLASS ANALYSIS

print("="*80)
print("PER-CLASS ANALYSIS")
print("="*80 + "\n")

def compute_confusion_matrix(y_true, y_pred, num_classes):
    cm = np.zeros((num_classes, num_classes))
    for i in range(len(y_true)):
        cm[y_true[i], y_pred[i]] += 1
    return cm

# Confusion matrices
cm_linear = compute_confusion_matrix(y_val_lp, y_val_pred, num_classes)
cm_mlp = compute_confusion_matrix(y_val_lp, mlp_val_pred, num_classes)
cm_svm = compute_confusion_matrix(y_val_lp, svm_val_pred, num_classes)

# Per-class accuracy
print("Per-Class Accuracy (MLP):\n")
per_class_acc_mlp = []
for c in range(num_classes):
    if np.sum(cm_mlp[c, :]) > 0:
        acc = cm_mlp[c, c] / np.sum(cm_mlp[c, :])
        per_class_acc_mlp.append(acc)
        print(f"  {class_names[c]:<30} {acc:.4f}")
    else:
        per_class_acc_mlp.append(0)

print(f"\nTop-5 Best Classes:")
top_5_idx = np.argsort(per_class_acc_mlp)[-5:][::-1]
for idx in top_5_idx:
    if idx < len(class_names):
        print(f"  {class_names[idx]:<30} {per_class_acc_mlp[idx]:.4f}")

print(f"\nBottom-5 Worst Classes:")
bottom_5_idx = np.argsort(per_class_acc_mlp)[:5]
for idx in bottom_5_idx:
    if idx < len(class_names):
        print(f"  {class_names[idx]:<30} {per_class_acc_mlp[idx]:.4f}")

print(f"\n" + "="*80)
print("✓ Per-class analysis complete\n")


PER-CLASS ANALYSIS

Per-Class Accuracy (MLP):

  BD30                           0.0603
  BD33                           0.0714
  BD39                           0.9032
  BD49                           0.3178
  BD51                           0.5437
  BD52                           0.2404
  BD56                           0.1207
  BD57                           0.0288
  BD70                           0.0714
  BD72                           0.3810
  BD75                           0.1111
  BD76                           0.7500
  BD79                           0.5532
  BD85                           0.0000
  BD87                           0.4348
  BD91                           0.4091
  BD93                           0.4536
  BD95                           0.6484
  BR22                           0.0526
  BR23                           0.8073
  BRRI102                        0.0625
  BRRI67                         0.0577
  BRRI74                         0.0000
  Binadhan10                     

# CELL 12: CLASSIFIER COMPARISON


In [20]:
# CELL 12: CLASSIFIER COMPARISON

print("="*80)
print("CLASSIFIER COMPARISON")
print("="*80 + "\n")

comparison_data = {
    'Linear Probe': {
        'Accuracy': linear_probe_acc,
        'Precision': linear_probe_precision,
        'Recall': linear_probe_recall,
        'F1': linear_probe_f1
    },
    'MLP': {
        'Accuracy': mlp_acc,
        'Precision': mlp_precision,
        'Recall': mlp_recall,
        'F1': mlp_f1
    },
    'SVM': {
        'Accuracy': svm_acc,
        'Precision': svm_precision,
        'Recall': svm_recall,
        'F1': svm_f1
    }
}

print("Metrics Comparison:\n")
print(f"{'Classifier':<15} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 70)
for clf_name, metrics in comparison_data.items():
    print(f"{clf_name:<15} {metrics['Accuracy']:<12.4f} {metrics['Precision']:<12.4f} {metrics['Recall']:<12.4f} {metrics['F1']:<12.4f}")

print("-" * 70)

best_clf = max(comparison_data.items(), key=lambda x: x[1]['Accuracy'])
print(f"\n✓ Best Classifier: {best_clf[0]} (Accuracy: {best_clf[1]['Accuracy']:.4f})\n")

print("="*80)
print("✓ Comparison complete\n")


CLASSIFIER COMPARISON

Metrics Comparison:

Classifier      Accuracy     Precision    Recall       F1-Score    
----------------------------------------------------------------------
Linear Probe    0.2887       0.2862       0.2862       0.2862      
MLP             0.3582       0.3598       0.3598       0.3598      
SVM             0.1079       0.1044       0.1044       0.1044      
----------------------------------------------------------------------

✓ Best Classifier: MLP (Accuracy: 0.3582)

✓ Comparison complete



# CELL 13: TRAINING LOSS VISUALIZATION

In [21]:
# CELL 13: TRAINING LOSS VISUALIZATION

print("="*80)
print("VISUALIZATION: TRAINING LOSS")
print("="*80 + "\n")

# Load saved losses
train_losses = np.load(os.path.join(OUTPUT_DIR, 'train_losses.npy'))

# Plot training loss
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
axes[0].plot(range(1, len(train_losses) + 1), train_losses, linewidth=2.5, color='#3498db', marker='o')
axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0].set_title('DINO Pretraining Loss', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=np.min(train_losses), color='r', linestyle='--', linewidth=2, label=f'Best: {np.min(train_losses):.4f}')
axes[0].legend(fontsize=11)

# Comparison bar chart
clf_names = list(comparison_data.keys())
accuracies = [comparison_data[clf]['Accuracy'] for clf in clf_names]
colors = ['#3498db', '#2ecc71', '#e74c3c']
bars = axes[1].bar(clf_names, accuracies, color=colors, edgecolor='black', linewidth=2)
axes[1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[1].set_title('Downstream Classifier Accuracy', fontsize=14, fontweight='bold')
axes[1].set_ylim([0, 1])
axes[1].grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_comparison.png'), dpi=150, bbox_inches='tight')
print(f"✓ Saved visualization to {OUTPUT_DIR}/training_comparison.png\n")
plt.close()

print("="*80)
print("✓ Visualization complete\n")


VISUALIZATION: TRAINING LOSS

✓ Saved visualization to /kaggle/working/training_comparison.png

✓ Visualization complete



# CELL 14: CONFUSION MATRIX HEATMAP

In [22]:
# CELL 14: CONFUSION MATRIX HEATMAP

print("="*80)
print("CONFUSION MATRIX VISUALIZATION")
print("="*80 + "\n")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

matrices = [cm_linear, cm_mlp, cm_svm]
titles = ['Linear Probe', 'MLP', 'SVM']
names = [y_val_pred, mlp_val_pred, svm_val_pred]

for idx, (cm, title, ax) in enumerate(zip(matrices, titles, axes)):
    # Normalize confusion matrix
    cm_norm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-8)
    
    # Plot heatmap
    im = ax.imshow(cm_norm, cmap='Blues', aspect='auto')
    ax.set_title(f'{title} Confusion Matrix', fontsize=12, fontweight='bold')
    ax.set_xlabel('Predicted', fontsize=11)
    ax.set_ylabel('True', fontsize=11)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Normalized Count', fontsize=10)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrices.png'), dpi=150, bbox_inches='tight')
print(f"✓ Saved confusion matrices to {OUTPUT_DIR}/confusion_matrices.png\n")
plt.close()

print("="*80)
print("✓ Confusion matrix visualization complete\n")


CONFUSION MATRIX VISUALIZATION

✓ Saved confusion matrices to /kaggle/working/confusion_matrices.png

✓ Confusion matrix visualization complete



# CELL 15: FINAL SUMMARY REPORT

In [23]:
# CELL 15: FINAL SUMMARY REPORT

print("\n" + "="*80)
print("DINO RICE LEAF DISEASE CLASSIFICATION - FINAL REPORT")
print("="*80 + "\n")

print("1. DATASET STATISTICS:")
print(f"   • Total images: {len(train_dataset)}")
print(f"   • Classes: {num_classes}")
print(f"   • Class names: {', '.join(class_names[:5])}{'...' if num_classes > 5 else ''}\n")

print("2. DINO PRETRAINING:")
print(f"   • Epochs trained: {len(train_losses)}")
print(f"   • Final loss: {train_losses[-1]:.4f}")
print(f"   • Best loss: {np.min(train_losses):.4f}")
print(f"   • Feature dimension: {train_features.shape[1]}")
print(f"   • Total training samples: {train_features.shape[0]}\n")

print("3. DOWNSTREAM EVALUATION (80-20 split):")
print(f"   • Training samples: {X_train_lp.shape[0]}")
print(f"   • Validation samples: {X_val_lp.shape[0]}\n")

print("4. LINEAR PROBE RESULTS:")
print(f"   ✓ Accuracy:  {linear_probe_acc:.4f}")
print(f"   ✓ Precision: {linear_probe_precision:.4f}")
print(f"   ✓ Recall:    {linear_probe_recall:.4f}")
print(f"   ✓ F1-Score:  {linear_probe_f1:.4f}\n")

print("5. MLP CLASSIFIER RESULTS:")
print(f"   ✓ Accuracy:  {mlp_acc:.4f}")
print(f"   ✓ Precision: {mlp_precision:.4f}")
print(f"   ✓ Recall:    {mlp_recall:.4f}")
print(f"   ✓ F1-Score:  {mlp_f1:.4f}\n")

print("6. SVM CLASSIFIER RESULTS:")
print(f"   ✓ Accuracy:  {svm_acc:.4f}")
print(f"   ✓ Precision: {svm_precision:.4f}")
print(f"   ✓ Recall:    {svm_recall:.4f}")
print(f"   ✓ F1-Score:  {svm_f1:.4f}\n")

print("7. BEST CLASSIFIER:")
print(f"   🏆 {best_clf[0]} with {best_clf[1]['Accuracy']:.4f} accuracy\n")

print("8. OUTPUT FILES:")
print(f"   ✓ Best checkpoint: {os.path.join(checkpoint_dir, 'best_checkpoint.pt')}")
print(f"   ✓ Training losses: {os.path.join(OUTPUT_DIR, 'train_losses.npy')}")
print(f"   ✓ Features: {os.path.join(OUTPUT_DIR, 'train_features.npy')}")
print(f"   ✓ Labels: {os.path.join(OUTPUT_DIR, 'train_labels.npy')}")
print(f"   ✓ Visualizations: {OUTPUT_DIR}/*.png\n")

print("="*80)
print("✓✓✓ PIPELINE COMPLETE - ALL ANALYSIS FINISHED ✓✓✓")
print("="*80 + "\n")



DINO RICE LEAF DISEASE CLASSIFICATION - FINAL REPORT

1. DATASET STATISTICS:
   • Total images: 19000
   • Classes: 38
   • Class names: BD30, BD33, BD39, BD49, BD51...

2. DINO PRETRAINING:
   • Epochs trained: 10
   • Final loss: -142.7732
   • Best loss: -143.6975
   • Feature dimension: 384
   • Total training samples: 19000

3. DOWNSTREAM EVALUATION (80-20 split):
   • Training samples: 15200
   • Validation samples: 3800

4. LINEAR PROBE RESULTS:
   ✓ Accuracy:  0.2887
   ✓ Precision: 0.2862
   ✓ Recall:    0.2862
   ✓ F1-Score:  0.2862

5. MLP CLASSIFIER RESULTS:
   ✓ Accuracy:  0.3582
   ✓ Precision: 0.3598
   ✓ Recall:    0.3598
   ✓ F1-Score:  0.3598

6. SVM CLASSIFIER RESULTS:
   ✓ Accuracy:  0.1079
   ✓ Precision: 0.1044
   ✓ Recall:    0.1044
   ✓ F1-Score:  0.1044

7. BEST CLASSIFIER:
   🏆 MLP with 0.3582 accuracy

8. OUTPUT FILES:
   ✓ Best checkpoint: /kaggle/working/checkpoints/best_checkpoint.pt
   ✓ Training losses: /kaggle/working/train_losses.npy
   ✓ Features: /k

# CELL 13: TRAINING LOSS & ACCURACY COMPARISON (INLINE)

In [24]:
# CELL 13: TRAINING LOSS & ACCURACY COMPARISON (INLINE)

print("="*80)
print("VISUALIZATION: TRAINING LOSS & CLASSIFIER COMPARISON")
print("="*80 + "\n")

# Load saved losses
train_losses = np.load(os.path.join(OUTPUT_DIR, 'train_losses.npy'))

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Training Loss Curve
axes[0, 0].plot(range(1, len(train_losses) + 1), train_losses, linewidth=3, 
                color='#3498db', marker='o', markersize=8, label='Training Loss')
axes[0, 0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Loss Value', fontsize=12, fontweight='bold')
axes[0, 0].set_title('DINO Pretraining Loss Curve', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, linestyle='--')
axes[0, 0].axhline(y=np.min(train_losses), color='r', linestyle='--', linewidth=2.5, 
                   label=f'Best Loss: {np.min(train_losses):.4f}')
axes[0, 0].legend(fontsize=11, loc='best')
axes[0, 0].fill_between(range(1, len(train_losses) + 1), train_losses, alpha=0.2, color='#3498db')

# 2. Classifier Accuracy Comparison
clf_names = list(comparison_data.keys())
accuracies = [comparison_data[clf]['Accuracy'] for clf in clf_names]
colors_bar = ['#3498db', '#2ecc71', '#e74c3c']
bars = axes[0, 1].bar(clf_names, accuracies, color=colors_bar, edgecolor='black', linewidth=2.5, alpha=0.8)
axes[0, 1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Downstream Classifier Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_ylim([0, 1])
axes[0, 1].grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                   f'{height:.4f}\n({height*100:.2f}%)', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

# 3. F1-Score Comparison
f1_scores = [comparison_data[clf]['F1'] for clf in clf_names]
bars2 = axes[1, 0].bar(clf_names, f1_scores, color=colors_bar, edgecolor='black', linewidth=2.5, alpha=0.8)
axes[1, 0].set_ylabel('F1-Score', fontsize=12, fontweight='bold')
axes[1, 0].set_title('F1-Score Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_ylim([0, 1])
axes[1, 0].grid(axis='y', alpha=0.3, linestyle='--')

for bar in bars2:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                   f'{height:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# 4. Precision vs Recall
precisions = [comparison_data[clf]['Precision'] for clf in clf_names]
recalls = [comparison_data[clf]['Recall'] for clf in clf_names]

x_pos = np.arange(len(clf_names))
width = 0.35

bars_prec = axes[1, 1].bar(x_pos - width/2, precisions, width, label='Precision', 
                           color='#9b59b6', edgecolor='black', linewidth=2, alpha=0.8)
bars_recall = axes[1, 1].bar(x_pos + width/2, recalls, width, label='Recall', 
                            color='#f39c12', edgecolor='black', linewidth=2, alpha=0.8)

axes[1, 1].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Precision vs Recall Comparison', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(clf_names)
axes[1, 1].set_ylim([0, 1])
axes[1, 1].legend(fontsize=11, loc='best')
axes[1, 1].grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print("✓ Visualization displayed inline\n")


VISUALIZATION: TRAINING LOSS & CLASSIFIER COMPARISON

✓ Visualization displayed inline



# CELL 14: CONFUSION MATRICES (INLINE - LARGE VIEW)

In [25]:
# CELL 14: CONFUSION MATRICES (INLINE - LARGE VIEW)

print("="*80)
print("CONFUSION MATRICES VISUALIZATION")
print("="*80 + "\n")

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

matrices = [cm_linear, cm_mlp, cm_svm]
titles = ['Linear Probe', 'MLP', 'SVM']
accs = [linear_probe_acc, mlp_acc, svm_acc]

for idx, (cm, title, ax, acc) in enumerate(zip(matrices, titles, axes, accs)):
    # Normalize confusion matrix
    cm_norm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-8)
    
    # Plot heatmap
    im = ax.imshow(cm_norm, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
    ax.set_title(f'{title} - Accuracy: {acc:.4f}', fontsize=13, fontweight='bold', pad=10)
    ax.set_xlabel('Predicted Class', fontsize=11, fontweight='bold')
    ax.set_ylabel('True Class', fontsize=11, fontweight='bold')
    
    # Set tick labels
    ax.set_xticks(range(num_classes))
    ax.set_yticks(range(num_classes))
    ax.set_xticklabels([f'C{i}' for i in range(num_classes)], fontsize=9)
    ax.set_yticklabels([f'C{i}' for i in range(num_classes)], fontsize=9)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Normalized Count', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Confusion matrices displayed inline\n")


CONFUSION MATRICES VISUALIZATION

✓ Confusion matrices displayed inline



# CELL 15: PER-CLASS PERFORMANCE HEATMAP (INLINE)

In [26]:
# CELL 15: PER-CLASS PERFORMANCE HEATMAP (INLINE)

print("="*80)
print("PER-CLASS PERFORMANCE ANALYSIS")
print("="*80 + "\n")

# Compute per-class metrics for each classifier
def compute_per_class_metrics(y_true, y_pred, num_classes):
    accuracy = []
    precision = []
    recall = []
    f1 = []
    
    for c in range(num_classes):
        mask_true = y_true == c
        mask_pred = y_pred == c
        
        if np.sum(mask_true) > 0:
            acc = np.mean(y_pred[mask_true] == y_true[mask_true])
            accuracy.append(acc)
            
            if np.sum(mask_pred) > 0:
                prec = np.sum((y_pred[mask_true] == c)) / np.sum(mask_pred)
                precision.append(prec)
            else:
                precision.append(0)
            
            rec = np.sum((y_pred[mask_true] == c)) / np.sum(mask_true)
            recall.append(rec)
            
            if precision[-1] + recall[-1] > 0:
                f1.append(2 * precision[-1] * recall[-1] / (precision[-1] + recall[-1]))
            else:
                f1.append(0)
        else:
            accuracy.append(0)
            precision.append(0)
            recall.append(0)
            f1.append(0)
    
    return np.array(accuracy), np.array(precision), np.array(recall), np.array(f1)

# Compute metrics
linear_acc, linear_prec, linear_rec, linear_f1 = compute_per_class_metrics(y_val_lp, y_val_pred, num_classes)
mlp_acc, mlp_prec, mlp_rec, mlp_f1 = compute_per_class_metrics(y_val_lp, mlp_val_pred, num_classes)
svm_acc, svm_prec, svm_rec, svm_f1 = compute_per_class_metrics(y_val_lp, svm_val_pred, num_classes)

# Create heatmap
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_list = [linear_acc, mlp_acc, svm_acc, mlp_f1]
titles_heatmap = ['Linear Probe - Accuracy', 'MLP - Accuracy', 'SVM - Accuracy', 'MLP - F1-Score']

for idx, (metric, title, ax) in enumerate(zip(metrics_list, titles_heatmap, axes.flat)):
    # Create heatmap data (reshape for visualization)
    heatmap_data = metric.reshape(-1, 1)
    
    im = ax.imshow(heatmap_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    ax.set_yticks(range(num_classes))
    ax.set_yticklabels([class_names[i] if i < len(class_names) else f'Class {i}' for i in range(num_classes)], fontsize=9)
    ax.set_xticks([0])
    ax.set_xticklabels(['Score'], fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold', pad=10)
    
    # Add value labels
    for i in range(num_classes):
        ax.text(0, i, f'{metric[i]:.3f}', ha='center', va='center', 
               color='white' if metric[i] < 0.5 else 'black', fontweight='bold', fontsize=10)
    
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Score', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Per-class heatmaps displayed inline\n")

# Print detailed per-class analysis
print("\nDETAILED PER-CLASS PERFORMANCE (MLP - Best Classifier):\n")
print(f"{'Class':<30} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 80)
for i in range(num_classes):
    class_name = class_names[i] if i < len(class_names) else f'Class {i}'
    print(f"{class_name:<30} {mlp_acc[i]:<12.4f} {mlp_prec[i]:<12.4f} {mlp_rec[i]:<12.4f} {mlp_f1[i]:<12.4f}")

print("\n" + "="*80)
print("✓ Per-class analysis complete\n")


PER-CLASS PERFORMANCE ANALYSIS

✓ Per-class heatmaps displayed inline


DETAILED PER-CLASS PERFORMANCE (MLP - Best Classifier):

Class                          Accuracy     Precision    Recall       F1-Score    
--------------------------------------------------------------------------------
BD30                           0.0603       0.1556       0.0603       0.0870      
BD33                           0.0714       0.0959       0.0714       0.0819      
BD39                           0.9032       0.7500       0.9032       0.8195      
BD49                           0.3178       0.2012       0.3178       0.2464      
BD51                           0.5437       0.2545       0.5437       0.3467      
BD52                           0.2404       0.2119       0.2404       0.2252      
BD56                           0.1207       0.2692       0.1207       0.1667      
BD57                           0.0288       0.2308       0.0288       0.0513      
BD70                           0.0714      

In [27]:
# CELL 16: FINAL SUMMARY WITH ALL METRICS (INLINE)

print("\n" + "="*80)
print("FINAL SUMMARY - DINO RICE LEAF DISEASE CLASSIFICATION")
print("="*80 + "\n")

# Summary statistics
print("📊 PRETRAINING SUMMARY:")
print(f"   • Epochs trained: {len(train_losses)}")
print(f"   • Initial loss: {train_losses[0]:.4f}")
print(f"   • Final loss: {train_losses[-1]:.4f}")
print(f"   • Best loss: {np.min(train_losses):.4f}")
print(f"   • Improvement: {abs(train_losses[-1] - train_losses[0]):.4f}\n")

print("📈 DATASET BREAKDOWN:")
print(f"   • Total images: {len(train_dataset)}")
print(f"   • Number of classes: {num_classes}")
print(f"   • Feature dimension: {train_features.shape[1]}")
print(f"   • Training samples (80%): {X_train_lp.shape[0]}")
print(f"   • Validation samples (20%): {X_val_lp.shape[0]}\n")

print("🏆 CLASSIFIER PERFORMANCE RANKINGS:\n")

# Rank by accuracy
ranking = sorted(comparison_data.items(), key=lambda x: x[1]['Accuracy'], reverse=True)
for idx, (name, metrics) in enumerate(ranking, 1):
    print(f"   {idx}. {name}")
    print(f"      ├─ Accuracy:  {metrics['Accuracy']:.4f} ({metrics['Accuracy']*100:.2f}%)")
    print(f"      ├─ Precision: {metrics['Precision']:.4f}")
    print(f"      ├─ Recall:    {metrics['Recall']:.4f}")
    print(f"      └─ F1-Score:  {metrics['F1']:.4f}\n")

print("🎯 BEST CLASSIFIER DETAILS:")
best_name, best_metrics = ranking[0]
print(f"   🏅 {best_name.upper()}")
print(f"      • Accuracy:  {best_metrics['Accuracy']:.4f}")
print(f"      • Precision: {best_metrics['Precision']:.4f}")
print(f"      • Recall:    {best_metrics['Recall']:.4f}")
print(f"      • F1-Score:  {best_metrics['F1']:.4f}\n")

# Top and bottom performing classes
print("📍 TOP 5 BEST PERFORMING CLASSES (MLP):\n")
top_5_idx = np.argsort(mlp_acc)[-5:][::-1]
for rank, idx in enumerate(top_5_idx, 1):
    class_name = class_names[idx] if idx < len(class_names) else f'Class {idx}'
    print(f"   {rank}. {class_name:<30} Accuracy: {mlp_acc[idx]:.4f}")

print("\n📍 TOP 5 WORST PERFORMING CLASSES (MLP):\n")
bottom_5_idx = np.argsort(mlp_acc)[:5]
for rank, idx in enumerate(bottom_5_idx, 1):
    class_name = class_names[idx] if idx < len(class_names) else f'Class {idx}'
    print(f"   {rank}. {class_name:<30} Accuracy: {mlp_acc[idx]:.4f}")

print("\n" + "="*80)
print("✅ PIPELINE COMPLETED SUCCESSFULLY")
print("="*80 + "\n")

# Create final summary visualization
fig, ax = plt.subplots(figsize=(12, 8))
ax.axis('off')

summary_text = f"""
DINO RICE LEAF DISEASE CLASSIFICATION - FINAL REPORT

PRETRAINING RESULTS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Model Architecture: Vision Transformer (DINO)
• Epochs Trained: {len(train_losses)}
• Final Loss: {train_losses[-1]:.4f}
• Best Loss: {np.min(train_losses):.4f}

DOWNSTREAM CLASSIFICATION RESULTS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. {ranking[0][0]:<20} - Accuracy: {ranking[0][1]['Accuracy']:.4f} ⭐ BEST
2. {ranking[1][0]:<20} - Accuracy: {ranking[1][1]['Accuracy']:.4f}
3. {ranking[2][0]:<20} - Accuracy: {ranking[2][1]['Accuracy']:.4f}

BEST CLASSIFIER METRICS (MLP)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Accuracy:  {best_metrics['Accuracy']:.4f} ({best_metrics['Accuracy']*100:.2f}%)
• Precision: {best_metrics['Precision']:.4f}
• Recall:    {best_metrics['Recall']:.4f}
• F1-Score:  {best_metrics['F1']:.4f}

DATASET STATISTICS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Total Images: {len(train_dataset)}
• Classes: {num_classes}
• Feature Dimension: {train_features.shape[1]}
• Train/Val Split: {X_train_lp.shape[0]} / {X_val_lp.shape[0]}
"""

ax.text(0.05, 0.95, summary_text, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='#ecf0f1', alpha=0.9, pad=1))

plt.tight_layout()
plt.show()

print("✓ Final summary report displayed\n")



FINAL SUMMARY - DINO RICE LEAF DISEASE CLASSIFICATION

📊 PRETRAINING SUMMARY:
   • Epochs trained: 10
   • Initial loss: -43.0315
   • Final loss: -142.7732
   • Best loss: -143.6975
   • Improvement: 99.7417

📈 DATASET BREAKDOWN:
   • Total images: 19000
   • Number of classes: 38
   • Feature dimension: 384
   • Training samples (80%): 15200
   • Validation samples (20%): 3800

🏆 CLASSIFIER PERFORMANCE RANKINGS:

   1. MLP
      ├─ Accuracy:  0.3582 (35.82%)
      ├─ Precision: 0.3598
      ├─ Recall:    0.3598
      └─ F1-Score:  0.3598

   2. Linear Probe
      ├─ Accuracy:  0.2887 (28.87%)
      ├─ Precision: 0.2862
      ├─ Recall:    0.2862
      └─ F1-Score:  0.2862

   3. SVM
      ├─ Accuracy:  0.1079 (10.79%)
      ├─ Precision: 0.1044
      ├─ Recall:    0.1044
      └─ F1-Score:  0.1044

🎯 BEST CLASSIFIER DETAILS:
   🏅 MLP
      • Accuracy:  0.3582
      • Precision: 0.3598
      • Recall:    0.3598
      • F1-Score:  0.3598

📍 TOP 5 BEST PERFORMING CLASSES (MLP):

   1. 

In [28]:
# CELL 13: TRAINING LOSS CURVE (INLINE)

print("="*80)
print("GRAPH 1: DINO PRETRAINING LOSS CURVE")
print("="*80 + "\n")

train_losses = np.load(os.path.join(OUTPUT_DIR, 'train_losses.npy'))

fig, ax = plt.subplots(figsize=(14, 7))

epochs = range(1, len(train_losses) + 1)
ax.plot(epochs, train_losses, linewidth=3.5, color='#3498db', marker='o', 
        markersize=10, markerfacecolor='#2980b9', markeredgewidth=2, label='Training Loss')
ax.fill_between(epochs, train_losses, alpha=0.3, color='#3498db')

ax.axhline(y=np.min(train_losses), color='r', linestyle='--', linewidth=2.5, 
           label=f'Best Loss: {np.min(train_losses):.4f}', alpha=0.8)

ax.set_xlabel('Epoch', fontsize=13, fontweight='bold')
ax.set_ylabel('Loss Value', fontsize=13, fontweight='bold')
ax.set_title('DINO Self-Supervised Pretraining Loss', fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)
ax.legend(fontsize=12, loc='best', framealpha=0.95)

plt.tight_layout()
plt.show()

print("✓ Graph displayed\n")


GRAPH 1: DINO PRETRAINING LOSS CURVE

✓ Graph displayed



In [30]:
# CELL 14: CLASSIFIER ACCURACY COMPARISON (INLINE)

print("="*80)
print("GRAPH 2: DOWNSTREAM CLASSIFIER ACCURACY COMPARISON")
print("="*80 + "\n")

clf_names = list(comparison_data.keys())
accuracies = [comparison_data[clf]['Accuracy'] for clf in clf_names]
colors = ['#3498db', '#2ecc71', '#e74c3c']

fig, ax = plt.subplots(figsize=(12, 7))

bars = ax.bar(clf_names, accuracies, color=colors, edgecolor='black', linewidth=2.5, 
              alpha=0.85, width=0.6)

ax.set_ylabel('Accuracy', fontsize=13, fontweight='bold')
ax.set_title('Downstream Classifier Accuracy Comparison', fontsize=15, fontweight='bold', pad=20)
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3, linestyle='--')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.03,
           f'{height:.4f}\n({height*100:.2f}%)', 
           ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Graph displayed\n")


GRAPH 2: DOWNSTREAM CLASSIFIER ACCURACY COMPARISON

✓ Graph displayed



In [31]:
# CELL 15: ALL METRICS COMPARISON (4-Subplot - INLINE)

print("="*80)
print("GRAPH 3: COMPREHENSIVE METRICS COMPARISON (4-Subplot)")
print("="*80 + "\n")

clf_names = list(comparison_data.keys())
accuracies = [comparison_data[clf]['Accuracy'] for clf in clf_names]
precisions = [comparison_data[clf]['Precision'] for clf in clf_names]
recalls = [comparison_data[clf]['Recall'] for clf in clf_names]
f1_scores = [comparison_data[clf]['F1'] for clf in clf_names]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
colors = ['#3498db', '#2ecc71', '#e74c3c']

# 1. Accuracy
bars1 = axes[0, 0].bar(clf_names, accuracies, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[0, 0].set_title('Accuracy', fontsize=13, fontweight='bold')
axes[0, 0].set_ylim([0, 1])
axes[0, 0].grid(axis='y', alpha=0.3)
for bar in bars1:
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

# 2. Precision
bars2 = axes[0, 1].bar(clf_names, precisions, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[0, 1].set_title('Precision', fontsize=13, fontweight='bold')
axes[0, 1].set_ylim([0, 1])
axes[0, 1].grid(axis='y', alpha=0.3)
for bar in bars2:
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

# 3. Recall
bars3 = axes[1, 0].bar(clf_names, recalls, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[1, 0].set_title('Recall', fontsize=13, fontweight='bold')
axes[1, 0].set_ylim([0, 1])
axes[1, 0].grid(axis='y', alpha=0.3)
for bar in bars3:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

# 4. F1-Score
bars4 = axes[1, 1].bar(clf_names, f1_scores, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[1, 1].set_title('F1-Score', fontsize=13, fontweight='bold')
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(axis='y', alpha=0.3)
for bar in bars4:
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Graph displayed\n")


GRAPH 3: COMPREHENSIVE METRICS COMPARISON (4-Subplot)

✓ Graph displayed



In [34]:
# CELL 16: CONFUSION MATRICES (3-Subplot - INLINE) - CORRECTED

print("="*80)
print("GRAPH 4: CONFUSION MATRICES (All 3 Classifiers)")
print("="*80 + "\n")

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

matrices = [cm_linear, cm_mlp, cm_svm]
titles = ['Linear Probe', 'MLP', 'SVM']

# Get scalar accuracy values correctly
accs = [
    np.mean(y_val_pred == y_val_lp),  # Linear probe
    np.mean(mlp_val_pred == y_val_lp),  # MLP
    np.mean(svm_val_pred == y_val_lp)   # SVM
]

for idx, (cm, title, ax, acc) in enumerate(zip(matrices, titles, axes, accs)):
    cm_norm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-8)
    
    im = ax.imshow(cm_norm, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
    ax.set_title(f'{title}\nAccuracy: {float(acc):.4f}', fontsize=12, fontweight='bold', pad=15)
    ax.set_xlabel('Predicted Class', fontsize=11, fontweight='bold')
    ax.set_ylabel('True Class', fontsize=11, fontweight='bold')
    
    ax.set_xticks(range(min(num_classes, 10)))
    ax.set_yticks(range(min(num_classes, 10)))
    ax.set_xticklabels([f'C{i}' for i in range(min(num_classes, 10))], fontsize=9)
    ax.set_yticklabels([f'C{i}' for i in range(min(num_classes, 10))], fontsize=9)
    
    cbar = plt.colorbar(im, ax=ax, fraction=0.046)
    cbar.set_label('Normalized', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Graph displayed\n")


GRAPH 4: CONFUSION MATRICES (All 3 Classifiers)

✓ Graph displayed



In [35]:
# CELL 17: PER-CLASS PERFORMANCE HEATMAP (INLINE)

print("="*80)
print("GRAPH 5: PER-CLASS PERFORMANCE ANALYSIS (MLP - Best Classifier)")
print("="*80 + "\n")

# Compute per-class metrics
def compute_per_class_metrics(y_true, y_pred, num_classes):
    accuracy, precision, recall, f1 = [], [], [], []
    
    for c in range(num_classes):
        mask_true = y_true == c
        mask_pred = y_pred == c
        
        if np.sum(mask_true) > 0:
            acc = np.mean(y_pred[mask_true] == y_true[mask_true])
            accuracy.append(acc)
            
            if np.sum(mask_pred) > 0:
                prec = np.sum((y_pred[mask_true] == c)) / np.sum(mask_pred)
            else:
                prec = 0
            precision.append(prec)
            
            rec = np.sum((y_pred[mask_true] == c)) / np.sum(mask_true)
            recall.append(rec)
            
            if prec + rec > 0:
                f1_score = 2 * prec * rec / (prec + rec)
            else:
                f1_score = 0
            f1.append(f1_score)
        else:
            accuracy.append(0)
            precision.append(0)
            recall.append(0)
            f1.append(0)
    
    return np.array(accuracy), np.array(precision), np.array(recall), np.array(f1)

mlp_acc_pc, mlp_prec_pc, mlp_rec_pc, mlp_f1_pc = compute_per_class_metrics(y_val_lp, mlp_val_pred, num_classes)

# Create heatmap of all metrics
fig, ax = plt.subplots(figsize=(14, 10))

metrics_data = np.column_stack([mlp_acc_pc, mlp_prec_pc, mlp_rec_pc, mlp_f1_pc])

im = ax.imshow(metrics_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

class_labels = [class_names[i] if i < len(class_names) else f'Class {i}' for i in range(num_classes)]
ax.set_yticks(range(num_classes))
ax.set_yticklabels(class_labels, fontsize=10)
ax.set_xticks(range(4))
ax.set_xticklabels(['Accuracy', 'Precision', 'Recall', 'F1-Score'], fontsize=11, fontweight='bold')
ax.set_title('Per-Class Performance Metrics (MLP)', fontsize=14, fontweight='bold', pad=15)

# Add text annotations
for i in range(num_classes):
    for j in range(4):
        text = ax.text(j, i, f'{metrics_data[i, j]:.3f}',
                      ha="center", va="center", color="black" if metrics_data[i, j] < 0.5 else "white",
                      fontweight='bold', fontsize=9)

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Score', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Graph displayed\n")


GRAPH 5: PER-CLASS PERFORMANCE ANALYSIS (MLP - Best Classifier)

✓ Graph displayed



In [36]:
# CELL 18: LOSS TREND + CLASSIFIER RANKING (INLINE)

print("="*80)
print("GRAPH 6: LOSS TREND + CLASSIFIER RANKING (Combined)")
print("="*80 + "\n")

fig = plt.figure(figsize=(16, 8))
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

# 1. Loss trend (large subplot)
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(range(1, len(train_losses) + 1), train_losses, linewidth=3, color='#3498db', 
         marker='o', markersize=8, label='Training Loss')
ax1.fill_between(range(1, len(train_losses) + 1), train_losses, alpha=0.2, color='#3498db')
ax1.axhline(y=np.min(train_losses), color='r', linestyle='--', linewidth=2.5, 
            label=f'Best: {np.min(train_losses):.4f}')
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax1.set_title('DINO Pretraining Loss Trend', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)

# 2. Accuracy ranking
ax2 = fig.add_subplot(gs[1, 0])
ranking = sorted(comparison_data.items(), key=lambda x: x[1]['Accuracy'], reverse=True)
rank_names = [name for name, _ in ranking]
rank_accs = [metrics['Accuracy'] for _, metrics in ranking]
colors_rank = ['gold', 'silver', '#CD7F32']  # Gold, Silver, Bronze
bars_rank = ax2.barh(rank_names, rank_accs, color=colors_rank, edgecolor='black', linewidth=2)
ax2.set_xlabel('Accuracy', fontsize=11, fontweight='bold')
ax2.set_title('Classifier Ranking', fontsize=12, fontweight='bold')
ax2.set_xlim([0, 1])
for i, (bar, acc) in enumerate(zip(bars_rank, rank_accs)):
    ax2.text(acc + 0.02, bar.get_y() + bar.get_height()/2, f'{acc:.4f}', 
            va='center', fontsize=11, fontweight='bold')

# 3. F1-Score comparison
ax3 = fig.add_subplot(gs[1, 1])
f1_scores = [comparison_data[clf]['F1'] for clf in comparison_data.keys()]
x_pos = np.arange(len(comparison_data))
bars_f1 = ax3.bar(x_pos, f1_scores, color=['#3498db', '#2ecc71', '#e74c3c'], 
                  edgecolor='black', linewidth=2, alpha=0.85)
ax3.set_xticks(x_pos)
ax3.set_xticklabels(list(comparison_data.keys()), fontsize=10)
ax3.set_ylabel('F1-Score', fontsize=11, fontweight='bold')
ax3.set_title('F1-Score Comparison', fontsize=12, fontweight='bold')
ax3.set_ylim([0, 1])
ax3.grid(axis='y', alpha=0.3)
for bar, f1 in zip(bars_f1, f1_scores):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{f1:.4f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Graph displayed\n")


GRAPH 6: LOSS TREND + CLASSIFIER RANKING (Combined)

✓ Graph displayed



In [38]:
# CELL 19: EXPORT RESULTS TO CSV - FIXED

print("="*80)
print("EXPORTING RESULTS TO CSV")
print("="*80 + "\n")

# Get best classifier info
best_clf_name = max(comparison_data.items(), key=lambda x: x[1]['Accuracy'])[0]
best_clf_accuracy = max(comparison_data.items(), key=lambda x: x[1]['Accuracy'])[1]['Accuracy']

# 1. Overall Classifier Comparison CSV
comparison_df = pd.DataFrame({
    'Classifier': list(comparison_data.keys()),
    'Accuracy': [comparison_data[clf]['Accuracy'] for clf in comparison_data.keys()],
    'Precision': [comparison_data[clf]['Precision'] for clf in comparison_data.keys()],
    'Recall': [comparison_data[clf]['Recall'] for clf in comparison_data.keys()],
    'F1-Score': [comparison_data[clf]['F1'] for clf in comparison_data.keys()],
})

comparison_csv = os.path.join(OUTPUT_DIR, 'classifier_comparison.csv')
comparison_df.to_csv(comparison_csv, index=False)
print(f"✓ Saved: classifier_comparison.csv\n")
print(comparison_df.to_string(index=False))
print()

# 2. Per-Class Performance (MLP) CSV
mlp_acc_pc, mlp_prec_pc, mlp_rec_pc, mlp_f1_pc = compute_per_class_metrics(y_val_lp, mlp_val_pred, num_classes)

per_class_df = pd.DataFrame({
    'Class_Name': [class_names[i] if i < len(class_names) else f'Class {i}' for i in range(num_classes)],
    'Class_ID': range(num_classes),
    'Accuracy': mlp_acc_pc,
    'Precision': mlp_prec_pc,
    'Recall': mlp_rec_pc,
    'F1-Score': mlp_f1_pc,
})

per_class_csv = os.path.join(OUTPUT_DIR, 'per_class_performance.csv')
per_class_df.to_csv(per_class_csv, index=False)
print(f"✓ Saved: per_class_performance.csv\n")
print(per_class_df.to_string(index=False))
print()

# 3. Training Summary CSV
training_summary_df = pd.DataFrame({
    'Metric': [
        'Total Images',
        'Number of Classes',
        'Feature Dimension',
        'Epochs Trained',
        'Initial Loss',
        'Final Loss',
        'Best Loss',
        'Training Samples (80%)',
        'Validation Samples (20%)',
        'Best Classifier',
        'Best Accuracy',
    ],
    'Value': [
        len(train_dataset),
        num_classes,
        train_features.shape[1],
        len(train_losses),
        f'{train_losses[0]:.4f}',
        f'{train_losses[-1]:.4f}',
        f'{np.min(train_losses):.4f}',
        X_train_lp.shape[0],
        X_val_lp.shape[0],
        best_clf_name,
        f'{best_clf_accuracy:.4f}',
    ]
})

training_csv = os.path.join(OUTPUT_DIR, 'training_summary.csv')
training_summary_df.to_csv(training_csv, index=False)
print(f"✓ Saved: training_summary.csv\n")
print(training_summary_df.to_string(index=False))
print()

# 4. Training Loss by Epoch CSV
loss_df = pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Loss': train_losses,
})

loss_csv = os.path.join(OUTPUT_DIR, 'training_loss_by_epoch.csv')
loss_df.to_csv(loss_csv, index=False)
print(f"✓ Saved: training_loss_by_epoch.csv\n")
print(f"Loss data: {len(train_losses)} epochs")
print()

print("="*80)
print("✅ ALL CSV FILES EXPORTED SUCCESSFULLY")
print("="*80 + "\n")

print("📊 SUMMARY:")
print(f"   • classifier_comparison.csv (3 classifiers, 5 metrics)")
print(f"   • per_class_performance.csv ({num_classes} classes, 5 metrics)")
print(f"   • training_summary.csv (11 key metrics)")
print(f"   • training_loss_by_epoch.csv ({len(train_losses)} epochs)\n")

print("Location: /kaggle/working/\n")


EXPORTING RESULTS TO CSV

✓ Saved: classifier_comparison.csv

  Classifier  Accuracy  Precision   Recall  F1-Score
Linear Probe  0.288684   0.286236 0.286236  0.286236
         MLP  0.358158   0.359819 0.359819  0.359819
         SVM  0.107895   0.104425 0.104425  0.104425

✓ Saved: per_class_performance.csv

Class_Name  Class_ID  Accuracy  Precision   Recall  F1-Score
      BD30         0  0.060345   0.155556 0.060345  0.086957
      BD33         1  0.071429   0.095890 0.071429  0.081871
      BD39         2  0.903226   0.750000 0.903226  0.819512
      BD49         3  0.317757   0.201183 0.317757  0.246377
      BD51         4  0.543689   0.254545 0.543689  0.346749
      BD52         5  0.240385   0.211864 0.240385  0.225225
      BD56         6  0.120690   0.269231 0.120690  0.166667
      BD57         7  0.028846   0.230769 0.028846  0.051282
      BD70         8  0.071429   0.280000 0.071429  0.113821
      BD72         9  0.380952   0.533333 0.380952  0.444444
      BD75        

In [39]:
# CELL 20: FINAL SUMMARY REPORT (INLINE TEXT)

print("\n" + "="*80)
print("FINAL COMPREHENSIVE REPORT - DINO RICE LEAF DISEASE CLASSIFICATION")
print("="*80 + "\n")

ranking = sorted(comparison_data.items(), key=lambda x: x[1]['Accuracy'], reverse=True)

print("📊 PRETRAINING SUMMARY:")
print(f"   • Model: Vision Transformer (DINO Self-Supervised)")
print(f"   • Epochs: {len(train_losses)}")
print(f"   • Initial Loss: {train_losses[0]:.4f}")
print(f"   • Best Loss: {np.min(train_losses):.4f} (Epoch {np.argmin(train_losses) + 1})")
print(f"   • Final Loss: {train_losses[-1]:.4f}")
print(f"   • Convergence: ✅ Successful\n")

print("🎯 DATASET STATISTICS:")
print(f"   • Total Images: {len(train_dataset)}")
print(f"   • Classes: {num_classes}")
print(f"   • Feature Dimension: {train_features.shape[1]}")
print(f"   • Training Split: {X_train_lp.shape[0]} samples (80%)")
print(f"   • Validation Split: {X_val_lp.shape[0]} samples (20%)\n")

print("🏆 CLASSIFIER PERFORMANCE RANKING:\n")
for idx, (name, metrics) in enumerate(ranking, 1):
    medal = '🥇' if idx == 1 else '🥈' if idx == 2 else '🥉'
    print(f"   {medal} #{idx}. {name.upper()}")
    print(f"      ├─ Accuracy:  {metrics['Accuracy']:.4f} ({metrics['Accuracy']*100:.2f}%)")
    print(f"      ├─ Precision: {metrics['Precision']:.4f}")
    print(f"      ├─ Recall:    {metrics['Recall']:.4f}")
    print(f"      └─ F1-Score:  {metrics['F1']:.4f}\n")

print("⭐ BEST CLASSIFIER:")
best_name, best_metrics = ranking[0]
print(f"   🏅 {best_name}")
print(f"      • Accuracy: {best_metrics['Accuracy']:.4f} ({best_metrics['Accuracy']*100:.2f}%)")
print(f"      • Precision: {best_metrics['Precision']:.4f}")
print(f"      • Recall: {best_metrics['Recall']:.4f}")
print(f"      • F1-Score: {best_metrics['F1']:.4f}\n")

print("📈 TOP 10 BEST PERFORMING CLASSES:")
mlp_acc_pc, mlp_prec_pc, mlp_rec_pc, mlp_f1_pc = compute_per_class_metrics(y_val_lp, mlp_val_pred, num_classes)
top_10_idx = np.argsort(mlp_acc_pc)[-10:][::-1]
for rank, idx in enumerate(top_10_idx, 1):
    class_name = class_names[idx] if idx < len(class_names) else f'Class {idx}'
    print(f"   {rank:2d}. {class_name:<30} Accuracy: {mlp_acc_pc[idx]:.4f}")

print("\n📉 BOTTOM 5 WORST PERFORMING CLASSES:")
bottom_5_idx = np.argsort(mlp_acc_pc)[:5]
for rank, idx in enumerate(bottom_5_idx, 1):
    class_name = class_names[idx] if idx < len(class_names) else f'Class {idx}'
    print(f"   {rank}. {class_name:<30} Accuracy: {mlp_acc_pc[idx]:.4f}")

print(f"\n📁 OUTPUT FILES GENERATED:")
print(f"   ✓ Checkpoints: {checkpoint_dir}/best_checkpoint.pt")
print(f"   ✓ Features: {OUTPUT_DIR}/train_features.npy")
print(f"   ✓ Labels: {OUTPUT_DIR}/train_labels.npy")
print(f"   ✓ Loss: {OUTPUT_DIR}/train_losses.npy")
print(f"   ✓ CSVs: {OUTPUT_DIR}/*.csv\n")

print("="*80)
print("✅ PIPELINE COMPLETED SUCCESSFULLY")
print("="*80 + "\n")



FINAL COMPREHENSIVE REPORT - DINO RICE LEAF DISEASE CLASSIFICATION

📊 PRETRAINING SUMMARY:
   • Model: Vision Transformer (DINO Self-Supervised)
   • Epochs: 10
   • Initial Loss: -43.0315
   • Best Loss: -143.6975 (Epoch 3)
   • Final Loss: -142.7732
   • Convergence: ✅ Successful

🎯 DATASET STATISTICS:
   • Total Images: 19000
   • Classes: 38
   • Feature Dimension: 384
   • Training Split: 15200 samples (80%)
   • Validation Split: 3800 samples (20%)

🏆 CLASSIFIER PERFORMANCE RANKING:

   🥇 #1. MLP
      ├─ Accuracy:  0.3582 (35.82%)
      ├─ Precision: 0.3598
      ├─ Recall:    0.3598
      └─ F1-Score:  0.3598

   🥈 #2. LINEAR PROBE
      ├─ Accuracy:  0.2887 (28.87%)
      ├─ Precision: 0.2862
      ├─ Recall:    0.2862
      └─ F1-Score:  0.2862

   🥉 #3. SVM
      ├─ Accuracy:  0.1079 (10.79%)
      ├─ Precision: 0.1044
      ├─ Recall:    0.1044
      └─ F1-Score:  0.1044

⭐ BEST CLASSIFIER:
   🏅 MLP
      • Accuracy: 0.3582 (35.82%)
      • Precision: 0.3598
      • Recall

In [40]:
# CELL 13-24: COMPLETE VISUALIZATION & EXPORT PIPELINE

print("\n" + "="*100)
print("COMPLETE VISUALIZATION & EXPORT PIPELINE")
print("="*100 + "\n")

# ======================= HELPER FUNCTION =======================
def compute_per_class_metrics(y_true, y_pred, num_classes):
    accuracy, precision, recall, f1 = [], [], [], []
    
    for c in range(num_classes):
        mask_true = y_true == c
        mask_pred = y_pred == c
        
        if np.sum(mask_true) > 0:
            acc = np.mean(y_pred[mask_true] == y_true[mask_true])
            accuracy.append(acc)
            
            if np.sum(mask_pred) > 0:
                prec = np.sum((y_pred[mask_true] == c)) / np.sum(mask_pred)
            else:
                prec = 0
            precision.append(prec)
            
            rec = np.sum((y_pred[mask_true] == c)) / np.sum(mask_true)
            recall.append(rec)
            
            if prec + rec > 0:
                f1_score = 2 * prec * rec / (prec + rec)
            else:
                f1_score = 0
            f1.append(f1_score)
        else:
            accuracy.append(0)
            precision.append(0)
            recall.append(0)
            f1.append(0)
    
    return np.array(accuracy), np.array(precision), np.array(recall), np.array(f1)

# ======================= LOAD DATA =======================
train_losses = np.load(os.path.join(OUTPUT_DIR, 'train_losses.npy'))
mlp_acc_pc, mlp_prec_pc, mlp_rec_pc, mlp_f1_pc = compute_per_class_metrics(y_val_lp, mlp_val_pred, num_classes)

print("\n" + "-"*100)
print("GRAPH 1: TRAINING LOSS CURVE - DISPLAY + SAVE PNG")
print("-"*100 + "\n")

fig, ax = plt.subplots(figsize=(14, 8))
epochs = range(1, len(train_losses) + 1)
ax.plot(epochs, train_losses, linewidth=4, color='#3498db', marker='o', 
        markersize=10, markerfacecolor='#2980b9', markeredgewidth=2.5, label='Training Loss')
ax.fill_between(epochs, train_losses, alpha=0.25, color='#3498db')
ax.axhline(y=np.min(train_losses), color='r', linestyle='--', linewidth=3, 
           label=f'Best Loss: {np.min(train_losses):.4f}', alpha=0.8)
ax.set_xlabel('Epoch', fontsize=14, fontweight='bold')
ax.set_ylabel('Loss Value', fontsize=14, fontweight='bold')
ax.set_title('DINO Self-Supervised Pretraining Loss Curve', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.4, linestyle='--', linewidth=1)
ax.legend(fontsize=13, loc='best', framealpha=0.98)
plt.tight_layout()
plt.show()
png_path = os.path.join(OUTPUT_DIR, '01_training_loss_curve.png')
fig.savefig(png_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved PNG: {png_path}")
plt.close()

print("\n" + "-"*100)
print("GRAPH 2: CLASSIFIER ACCURACY COMPARISON - DISPLAY + SAVE PNG")
print("-"*100 + "\n")

clf_names = list(comparison_data.keys())
accuracies = [comparison_data[clf]['Accuracy'] for clf in clf_names]
colors = ['#3498db', '#2ecc71', '#e74c3c']

fig, ax = plt.subplots(figsize=(12, 7))
bars = ax.bar(clf_names, accuracies, color=colors, edgecolor='black', linewidth=2.5, 
              alpha=0.85, width=0.6)
ax.set_ylabel('Accuracy', fontsize=13, fontweight='bold')
ax.set_title('Downstream Classifier Accuracy Comparison', fontsize=15, fontweight='bold', pad=20)
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3, linestyle='--')
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.03,
           f'{height:.4f}\n({height*100:.2f}%)', 
           ha='center', va='bottom', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()
png_path = os.path.join(OUTPUT_DIR, '02_classifier_accuracy.png')
fig.savefig(png_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved PNG: {png_path}")
plt.close()

print("\n" + "-"*100)
print("GRAPH 3: ALL METRICS COMPARISON (4-Subplot) - DISPLAY + SAVE PNG")
print("-"*100 + "\n")

precisions = [comparison_data[clf]['Precision'] for clf in clf_names]
recalls = [comparison_data[clf]['Recall'] for clf in clf_names]
f1_scores = [comparison_data[clf]['F1'] for clf in clf_names]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Accuracy
bars1 = axes[0, 0].bar(clf_names, accuracies, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[0, 0].set_title('Accuracy', fontsize=13, fontweight='bold')
axes[0, 0].set_ylim([0, 1])
axes[0, 0].grid(axis='y', alpha=0.3)
for bar in bars1:
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

# Precision
bars2 = axes[0, 1].bar(clf_names, precisions, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[0, 1].set_title('Precision', fontsize=13, fontweight='bold')
axes[0, 1].set_ylim([0, 1])
axes[0, 1].grid(axis='y', alpha=0.3)
for bar in bars2:
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

# Recall
bars3 = axes[1, 0].bar(clf_names, recalls, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[1, 0].set_title('Recall', fontsize=13, fontweight='bold')
axes[1, 0].set_ylim([0, 1])
axes[1, 0].grid(axis='y', alpha=0.3)
for bar in bars3:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

# F1-Score
bars4 = axes[1, 1].bar(clf_names, f1_scores, color=colors, edgecolor='black', linewidth=2, alpha=0.85)
axes[1, 1].set_title('F1-Score', fontsize=13, fontweight='bold')
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(axis='y', alpha=0.3)
for bar in bars4:
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.4f}', 
                   ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()
png_path = os.path.join(OUTPUT_DIR, '03_all_metrics_comparison.png')
fig.savefig(png_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved PNG: {png_path}")
plt.close()

print("\n" + "-"*100)
print("GRAPH 4: CONFUSION MATRICES (3 Classifiers) - DISPLAY + SAVE PNG")
print("-"*100 + "\n")

fig, axes = plt.subplots(1, 3, figsize=(22, 7))
matrices = [cm_linear, cm_mlp, cm_svm]
titles = ['Linear Probe', 'MLP', 'SVM']
accs = [
    np.mean(y_val_pred == y_val_lp),
    np.mean(mlp_val_pred == y_val_lp),
    np.mean(svm_val_pred == y_val_lp)
]

for idx, (cm, title, ax, acc) in enumerate(zip(matrices, titles, axes, accs)):
    cm_norm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-8)
    
    im = ax.imshow(cm_norm, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
    ax.set_title(f'{title}\nAccuracy: {float(acc):.4f}', fontsize=13, fontweight='bold', pad=15)
    ax.set_xlabel('Predicted Class', fontsize=12, fontweight='bold')
    ax.set_ylabel('True Class', fontsize=12, fontweight='bold')
    
    ax.set_xticks(range(min(num_classes, 10)))
    ax.set_yticks(range(min(num_classes, 10)))
    ax.set_xticklabels([f'C{i}' for i in range(min(num_classes, 10))], fontsize=10)
    ax.set_yticklabels([f'C{i}' for i in range(min(num_classes, 10))], fontsize=10)
    
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Normalized Count', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()
png_path = os.path.join(OUTPUT_DIR, '04_confusion_matrices.png')
fig.savefig(png_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved PNG: {png_path}")
plt.close()

print("\n" + "-"*100)
print("GRAPH 5: PER-CLASS PERFORMANCE HEATMAP - DISPLAY + SAVE PNG")
print("-"*100 + "\n")

fig, ax = plt.subplots(figsize=(14, 10))
metrics_data = np.column_stack([mlp_acc_pc, mlp_prec_pc, mlp_rec_pc, mlp_f1_pc])

im = ax.imshow(metrics_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

class_labels = [class_names[i] if i < len(class_names) else f'Class {i}' for i in range(num_classes)]
ax.set_yticks(range(num_classes))
ax.set_yticklabels(class_labels, fontsize=10)
ax.set_xticks(range(4))
ax.set_xticklabels(['Accuracy', 'Precision', 'Recall', 'F1-Score'], fontsize=11, fontweight='bold')
ax.set_title('Per-Class Performance Metrics (MLP - Best Classifier)', fontsize=14, fontweight='bold', pad=15)

for i in range(num_classes):
    for j in range(4):
        text = ax.text(j, i, f'{metrics_data[i, j]:.3f}',
                      ha="center", va="center", 
                      color="white" if metrics_data[i, j] < 0.5 else "black",
                      fontweight='bold', fontsize=9)

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Score', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()
png_path = os.path.join(OUTPUT_DIR, '05_per_class_heatmap.png')
fig.savefig(png_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved PNG: {png_path}")
plt.close()

print("\n" + "-"*100)
print("GRAPH 6: COMBINED ANALYSIS (Loss + Ranking + Summary) - DISPLAY + SAVE PNG")
print("-"*100 + "\n")

fig = plt.figure(figsize=(18, 10))
gs = fig.add_gridspec(2, 2, hspace=0.35, wspace=0.3)

# Loss trend (large)
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(range(1, len(train_losses) + 1), train_losses, linewidth=3, color='#3498db', 
         marker='o', markersize=8, label='Training Loss')
ax1.fill_between(range(1, len(train_losses) + 1), train_losses, alpha=0.2, color='#3498db')
ax1.axhline(y=np.min(train_losses), color='r', linestyle='--', linewidth=2.5, 
            label=f'Best: {np.min(train_losses):.4f}')
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax1.set_title('DINO Pretraining Loss Trend', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)

# Classifier ranking
ax2 = fig.add_subplot(gs[1, 0])
ranking = sorted(comparison_data.items(), key=lambda x: x[1]['Accuracy'], reverse=True)
rank_names = [name for name, _ in ranking]
rank_accs = [metrics['Accuracy'] for _, metrics in ranking]
colors_rank = ['#FFD700', '#C0C0C0', '#CD7F32']
bars_rank = ax2.barh(rank_names, rank_accs, color=colors_rank, edgecolor='black', linewidth=2)
ax2.set_xlabel('Accuracy', fontsize=11, fontweight='bold')
ax2.set_title('🏆 Classifier Ranking', fontsize=12, fontweight='bold')
ax2.set_xlim([0, 1])
for i, (bar, acc) in enumerate(zip(bars_rank, rank_accs)):
    ax2.text(acc + 0.02, bar.get_y() + bar.get_height()/2, f'{acc:.4f}', 
            va='center', fontsize=11, fontweight='bold')

# F1-Score
ax3 = fig.add_subplot(gs[1, 1])
x_pos = np.arange(len(clf_names))
bars_f1 = ax3.bar(x_pos, f1_scores, color=['#3498db', '#2ecc71', '#e74c3c'], 
                  edgecolor='black', linewidth=2, alpha=0.85)
ax3.set_xticks(x_pos)
ax3.set_xticklabels(clf_names, fontsize=10)
ax3.set_ylabel('F1-Score', fontsize=11, fontweight='bold')
ax3.set_title('F1-Score Comparison', fontsize=12, fontweight='bold')
ax3.set_ylim([0, 1])
ax3.grid(axis='y', alpha=0.3)
for bar, f1 in zip(bars_f1, f1_scores):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{f1:.4f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()
png_path = os.path.join(OUTPUT_DIR, '06_combined_analysis.png')
fig.savefig(png_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved PNG: {png_path}")
plt.close()

print("\n" + "="*100)
print("EXPORTING CSV FILES")
print("="*100 + "\n")

# Get best classifier
best_clf_name = max(comparison_data.items(), key=lambda x: x[1]['Accuracy'])[0]
best_clf_accuracy = max(comparison_data.items(), key=lambda x: x[1]['Accuracy'])[1]['Accuracy']

# 1. Classifier Comparison CSV
comparison_df = pd.DataFrame({
    'Classifier': list(comparison_data.keys()),
    'Accuracy': [comparison_data[clf]['Accuracy'] for clf in comparison_data.keys()],
    'Precision': [comparison_data[clf]['Precision'] for clf in comparison_data.keys()],
    'Recall': [comparison_data[clf]['Recall'] for clf in comparison_data.keys()],
    'F1-Score': [comparison_data[clf]['F1'] for clf in comparison_data.keys()],
})

comparison_csv = os.path.join(OUTPUT_DIR, 'classifier_comparison.csv')
comparison_df.to_csv(comparison_csv, index=False)
print(f"✓ Saved CSV: classifier_comparison.csv\n")
print(comparison_df.to_string(index=False))
print()

# 2. Per-Class Performance CSV
per_class_df = pd.DataFrame({
    'Class_Name': [class_names[i] if i < len(class_names) else f'Class {i}' for i in range(num_classes)],
    'Class_ID': range(num_classes),
    'Accuracy': mlp_acc_pc,
    'Precision': mlp_prec_pc,
    'Recall': mlp_rec_pc,
    'F1-Score': mlp_f1_pc,
})

per_class_csv = os.path.join(OUTPUT_DIR, 'per_class_performance.csv')
per_class_df.to_csv(per_class_csv, index=False)
print(f"✓ Saved CSV: per_class_performance.csv\n")
print(f"Rows: {len(per_class_df)}\n")

# 3. Training Summary CSV
training_summary_df = pd.DataFrame({
    'Metric': [
        'Total Images',
        'Number of Classes',
        'Feature Dimension',
        'Epochs Trained',
        'Initial Loss',
        'Final Loss',
        'Best Loss',
        'Training Samples (80%)',
        'Validation Samples (20%)',
        'Best Classifier',
        'Best Accuracy',
    ],
    'Value': [
        len(train_dataset),
        num_classes,
        train_features.shape[1],
        len(train_losses),
        f'{train_losses[0]:.4f}',
        f'{train_losses[-1]:.4f}',
        f'{np.min(train_losses):.4f}',
        X_train_lp.shape[0],
        X_val_lp.shape[0],
        best_clf_name,
        f'{best_clf_accuracy:.4f}',
    ]
})

training_csv = os.path.join(OUTPUT_DIR, 'training_summary.csv')
training_summary_df.to_csv(training_csv, index=False)
print(f"✓ Saved CSV: training_summary.csv\n")
print(training_summary_df.to_string(index=False))
print()

# 4. Training Loss by Epoch CSV
loss_df = pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Loss': train_losses,
})

loss_csv = os.path.join(OUTPUT_DIR, 'training_loss_by_epoch.csv')
loss_df.to_csv(loss_csv, index=False)
print(f"✓ Saved CSV: training_loss_by_epoch.csv")
print(f"Epochs: {len(train_losses)}\n")

print("="*100)
print("FINAL SUMMARY REPORT")
print("="*100 + "\n")

print("📊 PRETRAINING SUMMARY:")
print(f"   • Model: Vision Transformer (DINO Self-Supervised)")
print(f"   • Epochs: {len(train_losses)}")
print(f"   • Initial Loss: {train_losses[0]:.4f}")
print(f"   • Best Loss: {np.min(train_losses):.4f} (Epoch {np.argmin(train_losses) + 1})")
print(f"   • Final Loss: {train_losses[-1]:.4f}\n")

print("🎯 DATASET STATISTICS:")
print(f"   • Total Images: {len(train_dataset)}")
print(f"   • Classes: {num_classes}")
print(f"   • Feature Dimension: {train_features.shape[1]}")
print(f"   • Training Split: {X_train_lp.shape[0]} samples (80%)")
print(f"   • Validation Split: {X_val_lp.shape[0]} samples (20%)\n")

print("🏆 CLASSIFIER PERFORMANCE RANKING:\n")
for idx, (name, metrics) in enumerate(ranking, 1):
    medal = '🥇' if idx == 1 else '🥈' if idx == 2 else '🥉'
    print(f"   {medal} #{idx}. {name.upper()}")
    print(f"      ├─ Accuracy:  {metrics['Accuracy']:.4f}")
    print(f"      ├─ Precision: {metrics['Precision']:.4f}")
    print(f"      ├─ Recall:    {metrics['Recall']:.4f}")
    print(f"      └─ F1-Score:  {metrics['F1']:.4f}\n")

print("📁 OUTPUT FILES GENERATED:")
print(f"   ✓ PNG Files (6):")
print(f"      - 01_training_loss_curve.png")
print(f"      - 02_classifier_accuracy.png")
print(f"      - 03_all_metrics_comparison.png")
print(f"      - 04_confusion_matrices.png")
print(f"      - 05_per_class_heatmap.png")
print(f"      - 06_combined_analysis.png")
print(f"   ✓ CSV Files (4):")
print(f"      - classifier_comparison.csv")
print(f"      - per_class_performance.csv")
print(f"      - training_summary.csv")
print(f"      - training_loss_by_epoch.csv\n")

print("="*100)
print("✅ COMPLETE PIPELINE EXECUTED SUCCESSFULLY")
print("="*100 + "\n")



COMPLETE VISUALIZATION & EXPORT PIPELINE


----------------------------------------------------------------------------------------------------
GRAPH 1: TRAINING LOSS CURVE - DISPLAY + SAVE PNG
----------------------------------------------------------------------------------------------------

✓ Saved PNG: /kaggle/working/01_training_loss_curve.png

----------------------------------------------------------------------------------------------------
GRAPH 2: CLASSIFIER ACCURACY COMPARISON - DISPLAY + SAVE PNG
----------------------------------------------------------------------------------------------------

✓ Saved PNG: /kaggle/working/02_classifier_accuracy.png

----------------------------------------------------------------------------------------------------
GRAPH 3: ALL METRICS COMPARISON (4-Subplot) - DISPLAY + SAVE PNG
----------------------------------------------------------------------------------------------------

✓ Saved PNG: /kaggle/working/03_all_metrics_comparison.pn